
# single-depot VRP-TW generalized among $k$ machines

- Single depot vehicle routing problem - time windows, generalized among k machines.

#### Preliminary unit testing and checking complexity level

In [1]:
### Good unit test for error-proofing optimization!!!

def validate_jobs(jobs, jobsno_k_sets):
    '''
    This verifies that the job list and their qualifications don't cause an error.
    '''
    # Find intersection of unqualified jobs
    intersection_jobsno = set(jobs).intersection(*jobsno_k_sets)
    
    # Check if any job unqualified for all
    if intersection_jobsno: # Raise error if unqualified job/s exist in all sets
        raise ValueError(f"At least one job is unqualified for all machines--job/s: {intersection_jobsno}")
    
    return None

# Example usage
jobs = {1, 2, 3, 4, 5, 6, 7}  # All jobs
jobsno_1 = {1, 2, 4}  # Unqualified jobs (JOBS NO) for machine 1
jobsno_2 = {6, 3}     # Unqualified jobs for machine 2

validate_jobs(jobs, [jobsno_1, jobsno_2])


In [2]:
import math

# Number of edges (E)
num_edges = lambda N: math.ceil(N * (N - 1) / 2)

# Number of tour possibilities (T)
num_tour_combinations = lambda N: int(math.factorial(N-1)/2) if N <= 8 else f"{math.factorial(N-1)/2:.1e}"

print(num_edges(4))
print(num_tour_combinations(4))

6
3


#### Used past Traveling Salesperson Problem (TSP) code from DS 745 as a basis with the VRPSolver class

In [7]:
# ### DS 745 TSP Code / Also made adjustments 10/9/2024 JRW

# import pyomo.environ as pyo
# import time  # Import the time module
# import multiprocessing # processor count

# class TSPSolver:
#     def __init__(self, cost_matrix, a, b, column_names=None, pre_solved_subtours=None):
#         self.cost_matrix = cost_matrix
#         self.a = a # arrival times (time window)
#         self.b = b # expiration times (time window)
#         self.column_names = column_names
#         self.pre_solved_subtours = pre_solved_subtours
#         self.model = None
#         self.result = None

#     def create_model(self, subtour_constraints=True):
#         # Create a concrete Pyomo model
#         self.model = pyo.ConcreteModel()

#         # Define the indexes (square)
#         n = len(self.cost_matrix)
#         self.model.N = pyo.RangeSet(n)
#         self.model.N_from1 = pyo.RangeSet(1, n) # verify this!
#         self.model.k = pyo.RangeSet(2) # FOR NOW,!!!!!!!!! SHOULDN'T BE MAGIC NUMBER


#         #### Parameters
#         # cost matrix c_ijk
#         self.model.c = pyo.Param(self.model.N, 
#                                  self.model.N, 
#                                  self.model.k, 
#                                  initialize=lambda model, i, j, k: self.cost_matrix[i - 1][j - 1][k - 1])

#         # arrival times (EXPECTED) a
#         self.model.a = pyo.Param(self.model.N,
#                                  initialize=lambda model, i: self.a[i - 1])

#         # expiration times (soft constraint) b
#         self.model.b = pyo.Param(self.model.N,
#                                  initialize=lambda model, i: self.b[i - 1])
        
#         #### Variables
#         # Real variables (time)
#         self.model.t = pyo.Var(self.model.N,
#                                within=pyo.PositiveReals)
#         self.model.t_ik = pyo.Var(self.model.N, 
#                                   self.model.k, 
#                                   within=pyo.PositiveReals)
        
#         # Decision variables x_ij (Jobs to visit)
#         self.model.x = pyo.Var(self.model.N, 
#                                self.model.N,
#                                self.model.k, 
#                                within=pyo.Binary)

#         # Indicator variable u_ik
#         self.model.u = pyo.Var(self.model.N,
#                                self.model.k,  
#                                within=pyo.Binary)
        
#         #### Objectives
#         def objective_rule(model): # w1 min-max + (1-w1) min-sim
#             return sum(model.x[i, j] * model.c[i, j] for i in model.N for j in model.N)

#         self.model.objective = pyo.Objective(rule=objective_rule, 
#                                              sense=pyo.minimize)

#         #### Constraints            
#         def constraint1_rule(model, N):
#             return sum(model.x[i, N] for i in model.N if i != N) == 1

#         self.model.const1 = pyo.Constraint(self.model.N, 
#                                            rule=constraint1_rule)

#         def constraint2_rule(model, N):
#             return sum(model.x[N, j] for j in model.N if j != N) == 1

#         self.model.const2 = pyo.Constraint(self.model.N, 
#                                            rule=constraint2_rule)

#         def constraint3_rule(model, i):
#             return model.x[i, i] == 0

#         self.model.const3 = pyo.Constraint(self.model.N, 
#                                            rule=constraint3_rule)

#         # Subtour constraints
#         def constraint4_rule(model, i, j):
#             if i != j:
#                 return model.u[i] - model.u[j] + model.x[i, j] * n <= n - 1
#             else:
#                 return pyo.Constraint.Skip

#         if subtour_constraints:
#             self.model.const4 = pyo.Constraint(self.model.U, 
#                                                self.model.N, 
#                                                rule=constraint4_rule)

#         # Pre-solved subtour constraints
#         def constraint5_rule(model, i, j):
#             constraints = []
#             if self.pre_solved_subtours:
#                 for subtour in self.pre_solved_subtours:
#                     if len(subtour) > 1:
#                         constraint_expr = sum(model.x[i, j] for i, j in zip(subtour, subtour[1:])) == len(subtour) - 1 # [a,b,c] goes x[a,b] + x[b,c] == 3 - 1
#                         constraints.append(constraint_expr)
#             return constraints

#         if self.pre_solved_subtours:
#             self.model.const5 = pyo.ConstraintList()
#             for i, constraint_expr in enumerate(constraint5_rule(self.model, None, None)):
#                 self.model.const5.add(constraint_expr)
        

#     def find_subtours(self):
#         subtours = []
#         remaining_cities = list(self.model.N)

#         while remaining_cities:
#             current_city = remaining_cities[0]
#             subtour = [current_city]

#             while True:
#                 next_city = next(j for j in self.model.N if self.model.x[current_city, j]() > 0)
#                 if next_city == subtour[0]:
#                     break
#                 subtour.append(next_city)
#                 remaining_cities.remove(next_city)
#                 current_city = next_city

#             if len(subtour) > 1:
#                 subtours.append(subtour)

#             remaining_cities = [city for city in remaining_cities if city not in subtour]

#         # Convert subtours to strings with column names
#         if self.column_names:
#             subtour_strings = ['-'.join(self.column_names[i - 1] for i in subtour) for subtour in subtours]
#         else:
#             subtour_strings = ['-'.join(map(str, subtour)) for subtour in subtours]

#         return subtour_strings

    
#     def solve(self, subtour_constraints=True, verbose=True, num_threads=multiprocessing.cpu_count()): # Get the number of available processors
#         start_time = time.time()
        
#         self.create_model(subtour_constraints)

#         # Specify the path to the cbc solver executable
#         #cbc_solver_path = 'C:\\Users\\jrwoo\\anaconda3\\Library\\bin\\cbc.exe'


#         # Solve the model using MILP solver
#         solver = pyo.SolverFactory('gurobi', 
#                                    #executable=cbc_solver_path, 
#                                    options={'threads': num_threads})  # use the number of processors available for multithreading (helps greatly during branch and bound tree searches)
           
#         self.result = solver.solve(self.model, tee=verbose)
        
#         # # print Decision Variables? 
#         # if verbose:
#         #     print("Decision Variables:")
#         #     for i in self.model.N:
#         #         #u_i = int(self.model.u[i].value)
#         #         #print(f"u[{i}] = {u_i}")
#         #         for j in self.model.N:
#         #             if i != j and int(self.model.x[i, j].value) != 0:
#         #                 x_ij = int(self.model.x[i, j].value)
#         #                 print(f"x[{i},{j}] = {x_ij}")

        
#         end_time = time.time()
#         self.solution_time = round(end_time - start_time,2)  # Calculate the solution time

#     def get_solution(self):
#         if self.result is None:
#             raise Exception("You need to call solve() before getting the solution.")

#         # Find subtours in the solution
#         subtours = self.find_subtours()

#         # Round the objective value to the nearest integer
#         rounded_objective = round(pyo.value(self.model.objective))

#         solution = {
#             "status": self.result.solver.status,
#             "objective_value": rounded_objective,
#             "decision_variables": self.model.x,
#             "subtours": subtours,
#             "solution_time": self.solution_time  # Add solution time to the result
#         }

#         return solution




#### Sequential MILP Model, Hyperparameterized for Bayesian Opt (11/13/2024)

- ChatGPT assisted with debugging and brainstorming for porting from CMPL to Pyomo.
- This code went through months of meticulous porting, debugging, and extensions added to it (such as handling `delta_start` and other parameterization). That code was deleted from this file to save space, but it is still locally on my computer.

#### Grabbing the best AI_LPT for warmstarting

In [1]:
import pandas as pd
import numpy as np

def get_best_AI_LPT(QueueSize, ProblemNumber, df_path='df_training_AI_LPT.pkl'):
    """
    Gets one row from the data based on QueueSize and ProblemNumber.
    
    Filters rows by QueueSize, ProblemNumber, and Deadweight. 
    Then, it finds the row with the lowest 'obj_value_relative_percent'. 
    If there are ties, it picks the row with the highest 'Sequencing_rule_best_pairings'. If there are more ties, it grabs the first row.
    
    Parameters:
    - QueueSize (int): Queue size to filter
    - ProblemNumber (int): Problem number to filter
    - df_path (str): Path to data file (default: 'df_training_AI_LPT.pkl')

    Returns:
    - DataFrame: One filtered row
    """
    # load DataFrame
    df_training = pd.read_pickle(df_path)  # load dataframe

    # apply mask for filters
    mask = df_training[(df_training['ProblemSet'] == str(QueueSize)) & 
                       (df_training['ProblemNumber'] == str(ProblemNumber)) & 
                       (df_training['Deadweight'] == False)]

    # filter for minimum value
    min_rows = mask[np.isclose(mask['obj_value_relative_percent'].values, 
                               np.min(mask['obj_value_relative_percent'].values))]

    # filter for maximum pairing value
    final_row = min_rows[np.isclose(min_rows['Sequencing_rule_best_pairings'].values, 
                                    np.max(min_rows['Sequencing_rule_best_pairings'].values))].head(1)  # select first row

    return final_row  # return one row

# # Usage example
# QueueSize, ProblemNumber = 75, 1
# result = get_best_AI_LPT(QueueSize, ProblemNumber)
# display(result)

# ## Finding the SolutionNumber to use
# SolutionNumber_WarmStart = result['SolutionNumber'].iloc[0]
# SolutionNumber_WarmStart

### Optimization MILP Model (Sequential Policies)

In [2]:
import pyomo.environ as pyo
from pyomo.common.errors import ApplicationError # error handling
import time  # Timing purposes
import numpy as np
import pandas as pd
import multiprocessing  # Processor count
import itertools # handling tupling well
import re # for reading solver output
import os # unique for solver log reading in parallel  (avoid kernel CRASHING)
import logging # suppress warnings


class VRPSolver:
    def __init__(self, cost_matrix, problem_df,
                 problemset_num, problem_num, solution_num=1, 
                 warm_start=None, w1=0.5, M_array=None, debug=True, hyperparams=None):
        
        # Typical parameters (core configuration)
        self.cost_matrix = cost_matrix
        self.problem_df = problem_df
        a = np.concatenate(([0], problem_df['a'].values))
        b = np.concatenate(([0], problem_df['b'].values))
        self.a = a
        self.b = b
        self.a_sim = np.copy(self.a)  # sim values
        self.b_sim = np.copy(self.b)  # sim values
        a_true = np.concatenate(([0], problem_df['a_true'].values))
        b_true = np.concatenate(([0], problem_df['b_true'].values))
        self.a_true = a_true
        self.b_true = b_true
        self.problemset_num = problemset_num
        self.problem_num = problem_num
        self.solution_num = solution_num
        self.w1 = w1  # Weight for the biobjective function
        self.warm_start = warm_start  # Warm start solutions
        self.debug = debug  # Debug mode for logging details
        self.first_iteration = True  # Flag for first iteration timing

        # Initialize M_array or use defaults
        n = cost_matrix.shape[0]
        self.M_array = M_array if M_array and len(M_array) == n else [10000] * n

        # Hyperparameters with defaults (user-defined and Gurobi-specific)
        hyperparams = hyperparams or {}

        # ----- permanent
        self.max_iterations = hyperparams.get("max_iterations", 1)  # Maximum solver iterations
        self.delta_step = hyperparams.get("delta_step", 1000)  # Step size for delta increments
        self.num_threads = hyperparams.get("num_threads", multiprocessing.cpu_count()-1)  # Number of threads for parallel processing
        
        # --- User-defined hyperparameters ---
        self.delta_start = hyperparams.get("delta_start", 0)  # Initial delta for time windows (between 0 and 3000 minutes)
        self.time_limit = hyperparams.get("time_limit", 30)  # Solver time limit in seconds
        self.symmetry_breaking = hyperparams.get("symmetry_breaking", True)  # Enable symmetry-breaking constraints

        # --- Gurobi-specific hyperparameters ---
        self.mip_focus = hyperparams.get("mip_focus", 0)  # MIP focus: 0=auto, 1=optimality, 2=feasibility, 3=balance
        self.cuts = hyperparams.get("cuts", 0)  # Cut generation: 0=auto, 1=minimal, 2=moderate, 3=aggressive
        self.heuristics = hyperparams.get("heuristics", 0.05)  # Fraction of time for primal heuristics (0.0 to 1.0)
        self.presolve = hyperparams.get("presolve", -1)  # Presolve: -1=auto, 0=off, 1=conservative, 2=aggressive
        self.var_branch = hyperparams.get("var_branch", -1)  # Variable branching strategy: -1=default, 0-1 for custom
        self.branch_dir = hyperparams.get("branch_dir", 0)  # Branching direction: -1=down, 0=default, 1=up
        self.nodefile_start = hyperparams.get("nodefile_start", 0.5)  # Node file memory threshold (GB)
        self.symmetry = hyperparams.get("symmetry", 1)  # Symmetry detection: 1=enabled, 0=disabled

        # Initialize placeholders
        self.model = None
        self.result = None
        self.solution_time = None
        self.solution_time_gurobi = None
        self.best_bound = None

        # Generate symmetry-breaking pairs if enabled
        if self.symmetry_breaking:
            self.symmetry_breaking_pairs = self.generate_symmetry_breaking_pairs()
        else:
            self.symmetry_breaking_pairs = {}

    def generate_symmetry_breaking_pairs(self, bigM=10000):
        """
        Generate job pairs to break symmetry in optimization for each machine's schedule.
        """
        # Assume that the cost_matrix's third dimension corresponds to machines
        k = self.cost_matrix.shape[2]
        machine_processing_columns = [f'Machine{k_idx+1}_ProcessMinutes' for k_idx in range(k)]

        k = len(machine_processing_columns)
        symmetry_breaking_pairs = {}

        # For each machine
        for machine_index, machine_column in enumerate(machine_processing_columns, start=1):  # 1-index based index
            # Replace bigM and None values with NaN
            self.problem_df[machine_column] = self.problem_df[machine_column].replace([bigM, None], np.nan)

            # Sort by recipe, arrival, and processing time (LPT)
            sorted_jobs = self.problem_df.sort_values(by=['Recipe', 'a', machine_column], ascending=[True, True, False])

            # Filter non-NA jobs
            valid_jobs = sorted_jobs[sorted_jobs[machine_column].notna()]

            job_pairs = []

            # Group by recipe and arrival
            for _, group in valid_jobs.groupby(['Recipe', 'a']):
                job_ids = (group.index + 1).tolist()  # Convert to 1-based indexing

                # Generate pairs where j > i
                job_pairs.extend((i, j) for i, j in itertools.pairwise(job_ids) if j > i)

            # Add pairs if present
            if job_pairs:
                symmetry_breaking_pairs[machine_index] = job_pairs

        if self.debug: print(f"Symmetry breaking pairs used: {symmetry_breaking_pairs}")

        return symmetry_breaking_pairs
    
    def create_model(self, time_window_delta):
        # Create a Pyomo model
        self.model = pyo.ConcreteModel()

        # Define the indexes
        n = self.cost_matrix.shape[0]  # Jobs 0 (depot), 1, 2, ..., n-1
        k = self.cost_matrix.shape[2]  # Number of machines

        # Sets for jobs and machines
        self.model.N = pyo.RangeSet(0, n - 1)  # Including the depot (0)
        self.model.Jobs = pyo.RangeSet(0, n - 1)  # Including the depot (0)
        self.model.JobsT = pyo.RangeSet(0, n)  # Including the depot (0)
        self.model.Jobs1toN = pyo.RangeSet(1, n - 1)  # Excluding the depot (0)
        self.model.k = pyo.RangeSet(1, k)  # Pyomo ranges are inclusive

        #### Parameters
        # Cost matrix c[i, j, k]
        def cost_matrix_init(model, i, j, k):
            # Adjust for zero-based indexing when setting the cost matrix
            return self.cost_matrix[i, j, k - 1]  # k-1 to adjust for Pyomo's 1-based indexing

        self.model.c = pyo.Param(self.model.N, self.model.N, self.model.k, initialize=cost_matrix_init)

        # Time windows a[i] and b[i] with time_window_delta for updated a_sim and b_sim
        self.model.a = pyo.Param(self.model.N, initialize=lambda model, i: self.a_sim[i])
        self.model.b = pyo.Param(self.model.N, initialize=lambda model, i: self.b_sim[i] + time_window_delta)

        # Big M parameter array
        def M_init(model, i):
            return self.M_array[i - 1]

        self.model.M = pyo.Param(self.model.N, initialize=M_init)

        #### Variables
        # Time variables
        self.model.t = pyo.Var(self.model.N, within=pyo.NonNegativeReals)
        self.model.t_ik = pyo.Var(self.model.JobsT, self.model.k, within=pyo.NonNegativeReals)

        # Decision variables x[i, j, k]
        self.model.x = pyo.Var(self.model.N, self.model.N, self.model.k, within=pyo.Binary)

        # Indicator variable u[i, k]
        self.model.u = pyo.Var(self.model.N, self.model.k, within=pyo.Binary)

        # Makespan variable
        self.model.y = pyo.Var(within=pyo.NonNegativeReals)

        #### Objective
        def min_max_rule(model):
            return model.y  # We minimize the max flow time, which is represented by a y (linearization of max function)

        # (2) Max Flow Time
        def makespan_constraint_rule(model, k): # y >= t[n, k] for all machines k
            return model.y >= model.t_ik[n, k]  # Adjusted to match 0-based indexing
        self.model.makespan_constraint = pyo.Constraint(self.model.k, rule=makespan_constraint_rule)

        # (3) Average Makespan
        def min_sum_rule(model):
            return sum(
                model.c[i, j, k] * model.x[i, j, k]
                for i in model.N for j in model.N for k in model.k
                if i != j and model.c[i, j, k] != 10000  # Exclude diagonal and impossible cost
            )
        
        # (1) Biobjective function
        self.model.objective = pyo.Objective(
            expr=self.w1 * min_max_rule(self.model) + (1 - self.w1) * min_sum_rule(self.model) / 2,
            sense=pyo.minimize
        )

        #### Constraints

        # (4) Closed-Loop Start
        def closed_loop_first_job_rule(model, i, k):
            return model.t[i] - model.c[0, i, k] * model.x[0, i, k] >= 0
        self.model.closed_loop_first = pyo.Constraint(self.model.Jobs1toN, self.model.k, rule=closed_loop_first_job_rule)

        # (5) Closed-Loop End
        def closed_loop_last_job_rule(model, i, k):
            return model.t[i] + model.c[i, 0, k] * model.x[i, 0, k] <= model.t_ik[n, k]
        self.model.closed_loop_last = pyo.Constraint(self.model.Jobs1toN, self.model.k, rule=closed_loop_last_job_rule)

        # (6) Subtour elimination constraints
        def subtour_elimination_2a_rule(model, i, j, k):
            if i != j:
                return (
                    model.t_ik[i, k] - model.t_ik[j, k]
                    + (model.b[i] * model.x[i, j, k] - model.a[j] * model.x[i, j, k])
                    + model.c[i, j, k] * model.x[i, j, k]
                    <= model.b[i] - model.a[j]
                )
            else:
                return pyo.Constraint.Skip
        self.model.subtour_elimination_2a = pyo.Constraint(self.model.Jobs1toN, self.model.Jobs1toN, self.model.k, rule=subtour_elimination_2a_rule)

        # (7) Incapable arcs: Additional constraint to set x[i, j, k] to zero if cost matrix value is bigM
        def set_known_zero_x_rule(model, i, j, k):
            if self.cost_matrix[i, j, k - 1] == 10000:  # Adjust for zero-based indexing in cost_matrix
                return model.x[i, j, k] == 0
            return pyo.Constraint.Skip
        self.model.set_known_zero_x = pyo.Constraint(self.model.N, self.model.N, self.model.k, rule=set_known_zero_x_rule)

        # (8) First Job Assignment
        def first_job_combined_rule(model, j, k):
            if j == k:
                return model.x[0, j, k] == 1
            else:
                return model.x[0, j, k] == 0
        self.model.first_job_combined = pyo.Constraint(self.model.N, self.model.k, rule=first_job_combined_rule)

        # (9) Last Job Assignment
        def last_job_for_all_machines_rule(model, k):
            return sum(model.x[i, 0, k] for i in model.Jobs1toN) == 1
        self.model.last_job_for_all_machines = pyo.Constraint(self.model.k, rule=last_job_for_all_machines_rule)

        # (10)
        def assignment_rule_from(model, j):
            return sum(model.x[i, j, k] for i in model.Jobs if i != j for k in model.k) == 1
        self.model.assignment_from = pyo.Constraint(self.model.Jobs1toN, rule=assignment_rule_from)

        # (11) Outgoing Assignments
        def assignment_rule_to(model, j):
            return sum(model.x[j, i, k] for i in model.Jobs if i != j for k in model.k) == 1
        self.model.assignment_to = pyo.Constraint(self.model.Jobs1toN, rule=assignment_rule_to)

        # (12) Self-Loop Elimination
        def no_self_tour_rule(model, i, k):
            return model.x[i, i, k] == 0
        self.model.no_self_tour = pyo.Constraint(self.model.Jobs1toN, self.model.k, rule=no_self_tour_rule)

        # (13) Time Windows
        def time_window_rule_lower(model, i):
            return model.a[i] <= model.t[i]
        self.model.time_window_lower = pyo.Constraint(self.model.Jobs1toN, rule=time_window_rule_lower)

        def time_window_rule_upper(model, i):
            return model.t[i] <= model.b[i]
        self.model.time_window_upper = pyo.Constraint(self.model.Jobs1toN, rule=time_window_rule_upper)

        # (14) Linking u and x variables
        def linking_u_x_rule(model, i, k):
            return sum(model.x[i, j, k] for j in model.Jobs if j != i) == model.u[i, k]
        self.model.linking_u_x = pyo.Constraint(self.model.Jobs, self.model.k, rule=linking_u_x_rule)

        # (15) Depot initialization
        def depot_initialization_rule(model):
            return model.t[0] == 0
        self.model.depot_initialization = pyo.Constraint(rule=depot_initialization_rule)

        # (16) Time linking: job start per machine to job start
        def big_m_time_linking_rule1(model, i, k):
            return model.t_ik[i, k] <= model.t[i]
        self.model.big_m_time_link1 = pyo.Constraint(self.model.Jobs, self.model.k, rule=big_m_time_linking_rule1)

        # (17a and 17b) Big M time linking
        def big_m_time_linking_rule2(model, i, k):
            return model.t_ik[i, k] >= model.t[i] - model.M[i] + model.M[i] * model.u[i, k]
        self.model.big_m_time_link2 = pyo.Constraint(self.model.Jobs, self.model.k, rule=big_m_time_linking_rule2)

        def big_m_time_linking_rule3(model, i, k):
            return model.t_ik[i, k] <= model.M[i] * model.u[i, k]
        self.model.big_m_time_link3 = pyo.Constraint(self.model.Jobs, self.model.k, rule=big_m_time_linking_rule3)

        # (18) Symmetry-Breaking (for those big and nasty problems)
        if self.symmetry_breaking_pairs: # Uses the LPT after the same Recipe and Arrival Time 
            for k, job_pairs in self.symmetry_breaking_pairs.items():   # For each machine
                for i, j in job_pairs:  # For each job pair (no adjustment for job indices)
                    self.model.add_component( # Add symmetry-breaking constraint using original indices
                        f"symmetry_breaking_{i}_{j}_machine_{k}",
                        #pyo.Constraint(expr=self.model.t_ik[i, k] * self.model.u[j, k] <= self.model.t_ik[j, k]) # quadratic
                        pyo.Constraint(expr=self.model.t[i] <= self.model.t[j]) # Simplified to real versus real
                    )

    
    def solve(self, tolerances=1e-13):
        start_time = time.time()
        
        # Solver setup
        solver = pyo.SolverFactory('gurobi')
        options = {
            'threads': self.num_threads,
            #'FeasibilityTol': tolerances,
            #'IntFeasTol': tolerances,
            #'OptimalityTol': tolerances,
            'TimeLimit': self.time_limit,
            'MIPFocus': self.mip_focus,
            'Cuts': self.cuts,
            'Heuristics': self.heuristics,
            'Presolve': self.presolve,
            'VarBranch': self.var_branch,
            'BranchDir': self.branch_dir,
            'NodefileStart': self.nodefile_start,
            'Symmetry': self.symmetry
        }
        
        self.apply_warm_start()
        
        # Suppress Pyomo warnings during solve
        pyomo_logger = logging.getLogger("pyomo.core")
        original_level = pyomo_logger.getEffectiveLevel()
        pyomo_logger.setLevel(logging.DEBUG if self.debug else logging.ERROR)

        try:
            # Solve the model without a log file or extra solver metrics
            self.result = solver.solve(
                self.model, 
                tee=self.debug, 
                symbolic_solver_labels=True,
                warmstart=True,
                options=options
            )

            # Capture solution time for the first iteration
            if self.first_iteration:
                self.solution_time = round(time.time() - start_time, 2)
                self.first_iteration = False

        except (ValueError, ApplicationError) as e:
            if self.debug: 
                print(f"[ERROR] Solver encountered an error: {e}")
            self.solution_time = None
            self.result = None

        finally:
            pyomo_logger.setLevel(original_level)

    def apply_warm_start(self):
        if self.warm_start is None:
            return  # No warm start
    
        for k in range(self.warm_start.shape[2]):
            warm_start_k = self.warm_start[:, :, k]  # Slice for machine k
            #print(f"[DEBUG] Applying warm start for machine {k+1}")
    
            for i, row in enumerate(warm_start_k):
                for j, value in enumerate(row):
                    self.model.x[i, j, k + 1].value = value  # Set warm start value
                    #if value == 1: print(f"[DEBUG] Setting x[{i},{j},{k+1}] = {value}")

    def add_cumulative_constraints(self, i, j, k):
        """Adds constraints to fix x[i, j, k]."""
        constraint_name = f"fix_x_{i}_{j}_{k}"  # Unique constraint name
        if not hasattr(self.model, constraint_name):
            # Add cumulative constraint
            self.model.add_component(
                constraint_name,
                pyo.Constraint(expr=self.model.x[i, j, k] == 1)
            )
            if self.debug:
                print(f"[DEBUG] Added cumulative constraint: x[{i},{j},{k}] = 1")
    
    def iterative_solving(self):
        time_window_delta = self.delta_start  # Start delta
        SIMTIME = np.unique(self.a_true)  # Unique SIMTIME values
        
        # Initialize solution trackers
        prev_t_ik = {}
        prev_x_ijk = {}
        
        cumulative_x_fixes = set()  # Track cumulative fixings
        # tolerance = 1e-8  # Binary tolerance
        
        for sim_time in SIMTIME:
            if self.debug:
                print(f"[DEBUG] Simulating for SIMTIME = {sim_time}")
    
            # Update simulated a and b based on SIMTIME
            mask = sim_time >= self.a_true
            self.a_sim = np.where(mask, self.a_true, self.a_sim)
            self.b_sim = np.where(mask, self.b_true, self.b_sim)
    
            # if self.debug:
            #     # Print a, b, a_sim, b_sim updates
            #     print(f"[DEBUG]         a    : {self.a}")
            #     print(f"[DEBUG] Updated a_sim: {self.a_sim}")
            #     print(f"[DEBUG]        a_true: {self.a_true}")
            #     print(f"[DEBUG]         b    : {self.b}")
            #     print(f"[DEBUG] Updated b_sim: {self.b_sim}")
            #     print(f"[DEBUG]        b_true: {self.b_true}")
    
            # Create and update model
            self.create_model(time_window_delta)
            
            # Apply cumulative constraints
            if self.debug:
                print("[DEBUG] Applying cumulative constraints from previous iterations:")
            for (i, j, k) in cumulative_x_fixes:
                self.add_cumulative_constraints(i, j, k)
            
            # Apply new fixings based on t_ik and SIMTIME
            if self.debug:
                print("[DEBUG] Applying new fixings based on t_ik values and SIMTIME:")
            for (i, k), t_value in prev_t_ik.items():
                if sim_time < t_value:
                    continue  # Skip if SIMTIME not reached
        
                x_values = prev_x_ijk.get((i, k), {})
                for j, x_value in x_values.items():
                    if x_value != 1:  # Check for exact equality with 1:
                        if self.debug: print(f"[DEBUG] Skipping x[{i},{j},{k}] as value {x_value} is not close enough to 1.")
                        continue  # Only fix values close to 1
    
                    # Apply cumulative constraint and track
                    self.add_cumulative_constraints(i, j, k)
                    cumulative_x_fixes.add((i, j, k))
            
            # Solve model
            self.solve()
            
            # Retrieve and print solution
            solution = self.get_solution()

            if self.result is None:
                if self.debug:
                    print("[ERROR] Solver returned no result. Aborting iterations.")
                break  # Exit loop if no result

            if self.debug:
                print(f"x_assignments: {solution['x_variables']}")
                print(f"t_assignments: {solution['t_variables']}")
    
            # Store solution if feasible or optimal
            if self.result.solver.termination_condition in [
                pyo.TerminationCondition.optimal,
                pyo.TerminationCondition.feasible, 
                pyo.TerminationCondition.maxTimeLimit,
            ]:
                # Track non-zero t_ik values
                prev_t_ik = {
                    (i, k): value
                    for (i, k), value in self.model.t_ik.get_values().items()
                }
        
                # Track x values close to 1
                prev_x_ijk = {}
                for (i, j, k), value in self.model.x.get_values().items():
                    if value == 1:  # Check for exact equality with 1 (bugs if using tolerances)
                        if (i, k) not in prev_x_ijk:
                            prev_x_ijk[(i, k)] = {}  # Initialize inner dict
                        prev_x_ijk[(i, k)][j] = value  # Store j and value in inner dict
                
                # Update warm_start with the current x values
                self.warm_start = np.array([
                    [[round(self.model.x[i, j, k + 1].value) for k in range(self.warm_start.shape[2])] ## added a round
                     for j in range(self.warm_start.shape[1])]
                    for i in range(self.warm_start.shape[0])
                ], dtype=int) # added an int to stem WARNINGS on fixing with binaries

                if self.debug:
                    print("[DEBUG] Solution saved for next warm start.")
            else:
                if self.debug:
                    print("[DEBUG] No feasible solution found.")
                break  # Exit loop if infeasible

    def _print_constraints(self, constraint_names=None):
        """Print specific constraints of the model."""
        if constraint_names:
            for name in constraint_names:
                constraint = getattr(self.model, name, None)
                if constraint is not None:
                    print(f"Constraint: {name}")
                    constraint.pprint()
                else:
                    print(f"[WARNING] Constraint '{name}' not found in the model.")
        else:
            # Print all constraints if no specific names provided
            for name, constraint in self.model.component_map(pyo.Constraint).items():
                print(f"Constraint: {name}")
                constraint.pprint()
    def get_solution(self, rounding_tolerance=8):
        if self.result is None or self.result.solver.termination_condition not in [
            pyo.TerminationCondition.optimal,
            pyo.TerminationCondition.maxTimeLimit,
        ]:
            return {
                "status": "Aborted",
                "objective_value": None,
                "solution_time": self.solution_time,
                "makespan": None,
                "x_variables": {},
                "t_variables": {},
                "decision_matrix": None
            }

        num_jobs = len(self.model.N)
        num_machines = len(self.model.k)
        decision_matrix = np.zeros((num_jobs, num_jobs, num_machines),
                                   dtype=int)

        solution = {
            "status": self.result.solver.termination_condition,
            "objective_value": round(pyo.value(self.model.objective), 2),
            "solution_time": self.solution_time,
            "makespan": int(pyo.value(self.model.y)),  # Ensures makespan is integer
            "x_variables": {},
            "t_variables": {},
            "decision_matrix": decision_matrix
        }

        # Populate and sort t_variables by k first, then by time (value)
        sorted_t_values = sorted(
            ((i, k, value) for (i, k), value in self.model.t_ik.get_values().items() if int(round(value)) >=0.5),
            key=lambda x: (x[1], x[2])  # Sort by k first, then by time value
        )
    
        # Store sorted t_ik values in solution
        for i, k, value in sorted_t_values:
            solution["t_variables"][f"t_ik[{i},{k}]"] = int(round(value))

        # Collect binary x values cast to int for stability
        sorted_x_values = sorted(
            ((i, j, k, int(round(value))) for (i, j, k), value in self.model.x.get_values().items() if round(value) >= 1),
            key=lambda x: (x[2], x[0], x[1])
        )

        for i, j, k, value in sorted_x_values:
            decision_matrix[i, j, k - 1] = value
            solution["x_variables"][f"x[{i},{j},{k}]"] = int(round(value))

        return solution

##### Debugging specific Problem Numbers

In [13]:
# # Preparing the data into matrices and whatnot
# #QueueSize, ProblemNumber = 25, np.random.randint(1,300+1)
# #QueueSize, ProblemNumber = 50, np.random.randint(1,150+1)
# #QueueSize, ProblemNumber = 75, np.random.randint(1,100+1)
# QueueSize, ProblemNumber = 10, 1
# #QueueSize, ProblemNumber = 7, 1

# print(f'QueueSize:{QueueSize}; ProblemNumber:{ProblemNumber}')

# # Load problem df
# #file_path = f'ProblemSets/7Queue_debugging_development'
# file_path = f'ProblemSets/Training_Queue{QueueSize}'
# problem_df = pd.read_pickle(f'{file_path}/Problem_{ProblemNumber}_jobs.pkl')

# # Load cost matrix
# cost_matrix = np.load(f'{file_path}/Problem_{ProblemNumber}_cost_matrix.npy')

# # Load warm start solution from AI_LPT DES simulation to start!
# ## Finds the  best EXPECTED SolutionNumber to use
# SolutionNumber_WarmStart = get_best_AI_LPT(QueueSize, ProblemNumber)['SolutionNumber'].iloc[0]
# solution_dict = pd.read_pickle(f'{file_path}/Solutions/AI_LPT/Problem_{ProblemNumber}_Solution_{SolutionNumber_WarmStart}.pkl') # more efficient to store a dict of the solutions
# warm_start_obj_value,  warm_start_transition_matrix = solution_dict['obj_value'], np.array(solution_dict['transition_matrix']) # and then grab all the elements needed from the pkl dict
# print(f"Providing MIP of {warm_start_obj_value}")

# # Initialize VRPSolver
# # Hyperparameters
# delta_start_map = { # queuesize to delta start
#     10: 1000,
#     25: 1500,
#     50: 2000,
#     75: 2000
# }
# delta_start = delta_start_map.get(QueueSize, 1000) # between 0 and 3000 minutes, integers
# runtime_seconds_max = 25 # between 0.25 and 30 seconds (assumed much less than 5 minutes for speed and simulating)
# num_threads = multiprocessing.cpu_count()-1


# solver = VRPSolver(
#     cost_matrix=cost_matrix,
#     problem_df=problem_df,
#     problemset_num=QueueSize,
#     problem_num=ProblemNumber,
#     warm_start=warm_start_transition_matrix,
#     debug=True,  # Keep debug visible in the init call
#     hyperparams={
#          "delta_start": 1685,  # Between 0 and 3000 minutes
#          "num_threads": num_threads,  # Adjusted based on available cores
#          "time_limit": runtime_seconds_max,  # Solver time limit in seconds
#          "symmetry_breaking": True,  # Enable symmetry-breaking constraints
#          "mip_focus": 2,  # MIP focus control # MIP focus: 0=auto, 1=optimality, 2=feasibility, 3=balance
#          "cuts": 3,  # Cut generation control # Cut generation: 0=auto, 1=minimal, 2=moderate, 3=aggressive
#          "heuristics":  0.7218608689267029,  # Fraction of time for primal heuristics  # Fraction of time for primal heuristics (0.0 to 1.0)
#          "presolve": 1  # Presolve phase # Presolve: -1=auto, 0=off, 1=conservative, 2=aggressive
#     }
# )
# # {'delta_start': 1685, 'symmetry_breaking': True, 'mip_focus': 2, 'cuts': 3, 'heuristics': 0.7218608689267029, 'presolve': 1}

# # Run optimization with warm start
# solver.iterative_solving()

# # Get the final solution after the iteration process
# solution = solver.get_solution()

# # Output solution
# print(f"Solution Status: {solution['status']}")
# print(f"Objective Value: {solution['objective_value']}")

### Stochastic Model (Lookahead)

In [3]:
class StochasticVRPSolver(VRPSolver):
    def __init__(self, *args, stochastic_parameters=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Default parameters for stochastic behavior
        default_parameters = {
            'nanmean': True,
            'min_difference_time': 5,
            'earliness': 30,
        }
        self.stochastic_parameters = {**default_parameters, **(stochastic_parameters or {})}

        if self.debug:  # Use the `self.debug` inherited from the parent class
            print(f"[DEBUG] Initialized StochasticVRPSolver with stochastic_parameters: {self.stochastic_parameters}")


    def _print_constraints_and_objective(self, constraint_names=None):
        """
        Print specific constraints of the model, and include the objective function.
        """
        # Print Objective Function(s)
        print("Objective Function(s):")
        for obj in self.model.component_objects(pyo.Objective, active=True):
            print(f"Objective Name: {obj.name}")
            print(f"Objective Expression: {obj.expr}")
            print("-" * 50)
        
        # Print Constraints
        print("Constraints:")
        if constraint_names:
            for name in constraint_names:
                constraint = getattr(self.model, name, None)
                if constraint is not None:
                    print(f"Constraint: {name}")
                    constraint.pprint()
                else:
                    print(f"[WARNING] Constraint '{name}' not found in the model.")
        else:
            # Print all constraints if no specific names provided
            for name, constraint in self.model.component_map(pyo.Constraint).items():
                print(f"Constraint: {name}")
                constraint.pprint()

    def create_model_stochastic(self, time_window_delta, a_scenarios, b_scenarios, scenarios):

        if self.debug: 
            print(f"a_scenarios (shape: {a_scenarios.shape})")
            print(f"b_scenarios (shape: {b_scenarios.shape})")
            print(f"scenarios: {scenarios}")
        
        # Create a Pyomo model
        self.model = pyo.ConcreteModel()

        # Define the indexes
        n = self.cost_matrix.shape[0]  # Jobs 0 (depot), 1, 2, ..., n-1
        k = self.cost_matrix.shape[2]  # Number of machines

        # Sets for jobs and machines
        self.model.N = pyo.RangeSet(0, n - 1)  # Including the depot (0)
        self.model.Jobs = pyo.RangeSet(0, n - 1)  # Including the depot (0)
        self.model.JobsT = pyo.RangeSet(0, n)  # Including the depot (0)
        self.model.Jobs1toN = pyo.RangeSet(1, n - 1)  # Excluding the depot (0)
        self.model.k = pyo.RangeSet(1, k)  # Pyomo ranges are inclusive

        #### Scenarios Set and Parameter (NEWW!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!)
        # Define a Pyomo set for scenarios
        self.model.w = pyo.Set(initialize=list(scenarios.keys()))
    
        # Define a Pyomo parameter for scenario probabilities
        self.model.p_w = pyo.Param(
            self.model.w, 
            initialize=scenarios, 
            within=pyo.UnitInterval  # Ensure probabilities are between 0 and 1
        )

        if self.debug: # Check if p_w is initialized correctly
            print("[DEBUG] Checking scenario probabilities (p_w):")
            for w in self.model.w:
                print(f"Scenario '{w}': Probability = {self.model.p_w[w]}")

        #### Parameters
        # Cost matrix c[i, j, k]
        def cost_matrix_init(model, i, j, k):
            # Adjust for zero-based indexing when setting the cost matrix
            return self.cost_matrix[i, j, k - 1]  # k-1 to adjust for Pyomo's 1-based indexing

        self.model.c = pyo.Param(self.model.N, self.model.N, self.model.k, initialize=cost_matrix_init)

        # Time windows a[i] and b[i] with time_window_delta for updated a_sim and b_sim (WITH W)
        if self.debug: 
            print(f"[DEBUG] a_w: {a_scenarios}")
            print(f"[DEBUG] b_w: {b_scenarios}")
        a_scenarios = np.transpose(a_scenarios)  # Now shape will be (n_jobs, n_scenarios)
        b_scenarios = np.transpose(b_scenarios)  # Now shape will be (n_jobs, n_scenarios)
        
        # Define Pyomo parameters with transformed arrays
        self.model.a_w = pyo.Param(
            self.model.N,
            self.model.w,
            initialize=lambda model, i, w: a_scenarios[i, list(self.model.w).index(w)]
        )
        self.model.b_w = pyo.Param(
            self.model.N,
            self.model.w,
            initialize=lambda model, i, w: b_scenarios[i, list(self.model.w).index(w)] + time_window_delta
        )
        if self.debug: # Check if p_w is initialized correctly
            print("[DEBUG] Checking scenario probabilities (p_w):")
            for w in self.model.w:
                print(f"Scenario '{w}': Probability = {self.model.p_w[w]}")

        
        if self.debug: # Extract and print a_w and b_w as NumPy arrays
            print("[DEBUG] Checking a_w and b_w initialization:")
            # Create empty NumPy arrays to hold the values
            a_w_array = np.zeros((len(self.model.w), len(self.model.N)))
            b_w_array = np.zeros((len(self.model.w), len(self.model.N)))
        
            # Populate the arrays from Pyomo parameters
            for idx_w, w in enumerate(self.model.w):
                for idx_i in self.model.N:
                    a_w_array[idx_w, idx_i] = self.model.a_w[idx_i, w]
                    b_w_array[idx_w, idx_i] = self.model.b_w[idx_i, w]
        
            # Print the arrays
            print("[DEBUG] a_w as a NumPy array:")
            print(a_w_array)
            print("[DEBUG] b_w as a NumPy array:")
            print(b_w_array)


        # Big M parameter array
        def M_init(model, i):
            return self.M_array[i - 1]

        self.model.M = pyo.Param(self.model.N, initialize=M_init)

        #### Variables
        # Time variables
        self.model.t_w = pyo.Var(self.model.N, self.model.w, within=pyo.NonNegativeReals)
        self.model.t_ikw = pyo.Var(self.model.JobsT, self.model.k, self.model.w, within=pyo.NonNegativeReals)

        # Decision variables x[i, j, k]
        self.model.x = pyo.Var(self.model.N, self.model.N, self.model.k, within=pyo.Binary)

        # Indicator variable u[i, k]
        self.model.u = pyo.Var(self.model.N, self.model.k, within=pyo.Binary)

        # Max Flowtime variable
        self.model.y_w = pyo.Var(self.model.w, within=pyo.NonNegativeReals)

        #### (1a) Objective (WITH W)
        def min_max_rule(model): # Sum of p_w * y_w over all scenarios w
            return sum(
                model.p_w[w] * model.y_w[w] 
                for w in model.w
            )
            
        # (2) Max Flow Time (WITH W)
        def makespan_constraint_rule(model, k, w): # y >= t[n, k, w] for all machines k
            return model.y_w[w] >= model.t_ikw[n, k, w]  # Adjusted to match 0-based indexing
        self.model.makespan_constraint = pyo.Constraint(self.model.k, self.model.w, rule=makespan_constraint_rule)

        # (3) Average Makespan
        def min_sum_rule(model):
            return sum(
                model.c[i, j, k] * model.x[i, j, k]
                for i in model.N for j in model.N for k in model.k
                if i != j and model.c[i, j, k] != 10000  # Exclude diagonal and impossible cost
            )
        
        # (1) Biobjective function
        self.model.objective = pyo.Objective(
            expr=self.w1 * min_max_rule(self.model) + (1 - self.w1) * min_sum_rule(self.model) / 2,
            sense=pyo.minimize
        )

        #### Constraints

        # (4) Closed-Loop Start  (WITH W)
        def closed_loop_first_job_rule(model, i, k, w):
            return model.t_w[i,w] - model.c[0, i, k] * model.x[0, i, k] >= 0
        self.model.closed_loop_first = pyo.Constraint(self.model.Jobs1toN, self.model.k, self.model.w, rule=closed_loop_first_job_rule)

        # (5) Closed-Loop End  (WITH W)
        def closed_loop_last_job_rule(model, i, k, w):
            return model.t_w[i,w] + model.c[i, 0, k] * model.x[i, 0, k] <= model.t_ikw[n, k, w]
        self.model.closed_loop_last = pyo.Constraint(self.model.Jobs1toN, self.model.k, self.model.w, rule=closed_loop_last_job_rule)

        # (6) Subtour elimination constraints  (WITH W)
        def subtour_elimination_2a_rule(model, i, j, k, w):
            if i != j:
                return (
                    model.t_ikw[i, k, w] - model.t_ikw[j, k, w]
                    + (model.b_w[i, w] * model.x[i, j, k] - model.a_w[j, w] * model.x[i, j, k])
                    + model.c[i, j, k] * model.x[i, j, k]
                    <= model.b_w[i, w] - model.a_w[j, w]
                )
            else:
                return pyo.Constraint.Skip
        self.model.subtour_elimination_2a = pyo.Constraint(self.model.Jobs1toN, self.model.Jobs1toN, self.model.k, self.model.w, rule=subtour_elimination_2a_rule)

        # (7) Incapable arcs: Additional constraint to set x[i, j, k] to zero if cost matrix value is bigM
        def set_known_zero_x_rule(model, i, j, k):
            if self.cost_matrix[i, j, k - 1] == 10000:  # Adjust for zero-based indexing in cost_matrix
                return model.x[i, j, k] == 0
            return pyo.Constraint.Skip
        self.model.set_known_zero_x = pyo.Constraint(self.model.N, self.model.N, self.model.k, rule=set_known_zero_x_rule)

        # (8) First Job Assignment
        def first_job_combined_rule(model, j, k):
            if j == k:
                return model.x[0, j, k] == 1
            else:
                return model.x[0, j, k] == 0
        self.model.first_job_combined = pyo.Constraint(self.model.N, self.model.k, rule=first_job_combined_rule)

        # (9) Last Job Assignment
        def last_job_for_all_machines_rule(model, k):
            return sum(model.x[i, 0, k] for i in model.Jobs1toN) == 1
        self.model.last_job_for_all_machines = pyo.Constraint(self.model.k, rule=last_job_for_all_machines_rule)

        # (10) Outgoing Assignments
        def assignment_rule_from(model, j):
            return sum(model.x[i, j, k] for i in model.Jobs if i != j for k in model.k) == 1
        self.model.assignment_from = pyo.Constraint(self.model.Jobs1toN, rule=assignment_rule_from)

        # (11) Incoming Assignments
        def assignment_rule_to(model, j):
            return sum(model.x[j, i, k] for i in model.Jobs if i != j for k in model.k) == 1
        self.model.assignment_to = pyo.Constraint(self.model.Jobs1toN, rule=assignment_rule_to)

        # (12) Self-Loop Elimination
        def no_self_tour_rule(model, i, k):
            return model.x[i, i, k] == 0
        self.model.no_self_tour = pyo.Constraint(self.model.Jobs1toN, self.model.k, rule=no_self_tour_rule)

        # (13) Time Windows  (WITH W)
        def time_window_rule_lower(model, i, w):
            return model.a_w[i, w] <= model.t_w[i, w]
        self.model.time_window_lower = pyo.Constraint(self.model.Jobs1toN, self.model.w, rule=time_window_rule_lower)

        def time_window_rule_upper(model, i, w):
            return model.t_w[i, w] <= model.b_w[i, w]
        self.model.time_window_upper = pyo.Constraint(self.model.Jobs1toN, self.model.w, rule=time_window_rule_upper)

        # (14) Linking u and x variables
        def linking_u_x_rule(model, i, k):
            return sum(model.x[i, j, k] for j in model.Jobs if j != i) == model.u[i, k]
        self.model.linking_u_x = pyo.Constraint(self.model.Jobs, self.model.k, rule=linking_u_x_rule)

        # (15) Depot initialization  (WITH W)
        def depot_initialization_rule(model, w):
            return model.t_w[0, w] == 0
        self.model.depot_initialization = pyo.Constraint(self.model.w, rule=depot_initialization_rule)

        # (16) Time linking: job start per machine to job start  (WITH W)
        def big_m_time_linking_rule1(model, i, k, w):
            return model.t_ikw[i, k, w] <= model.t_w[i, w]
        self.model.big_m_time_link1 = pyo.Constraint(self.model.Jobs, self.model.k, self.model.w, rule=big_m_time_linking_rule1)

        # (17a and 17b) Big M time linking  (WITH W)
        def big_m_time_linking_rule2(model, i, k, w):
            return model.t_ikw[i, k, w] >= model.t_w[i, w] - model.M[i] + model.M[i] * model.u[i, k]
        self.model.big_m_time_link2 = pyo.Constraint(self.model.Jobs, self.model.k, self.model.w, rule=big_m_time_linking_rule2)

        def big_m_time_linking_rule3(model, i, k, w):
            return model.t_ikw[i, k, w] <= model.M[i] * model.u[i, k]
        self.model.big_m_time_link3 = pyo.Constraint(self.model.Jobs, self.model.k, self.model.w, rule=big_m_time_linking_rule3)

        # (18) Symmetry-Breaking (for those big and nasty problems)  (WITH W)
        if self.symmetry_breaking_pairs: # Uses the LPT after the same Recipe and Arrival Time 
            for k, job_pairs in self.symmetry_breaking_pairs.items():   # For each machine
                for i, j in job_pairs:  # For each job pair (no adjustment for job indices)
                    for w in self.model.w:
                        self.model.add_component( # Add symmetry-breaking constraint using original indices
                            f"symmetry_breaking_{i}_{j}_machine_{k}_scenario_{w}",
                            #pyo.Constraint(expr=self.model.t_ik[i, k] * self.model.u[j, k] <= self.model.t_ik[j, k]) # quadratic
                            pyo.Constraint(expr=self.model.t_w[i, w] <= self.model.t_w[j, w]) # Simplified to real versus real
                        )

    def scenario_generating(self, sim_time, bigM= 10000):
        """
        Generates job scheduling scenarios based on arrival timing, recipe clusters, and processing times.

        Parameters:
        sim_time (int): Current simulation time.

        Returns:
        dict: Adjusted scenarios ('a', 'b'), whether deterministic, and cluster probabilities.
        """
        # Extract parameters from the stochastic_parameters dictionary
        nanmean = self.stochastic_parameters['nanmean']
        min_difference_time = self.stochastic_parameters['min_difference_time']
        earliness = self.stochastic_parameters['earliness']

        a, b, problem_df = self.a_sim, self.b_sim, self.problem_df.copy()

        # if self.debug:
        #     print(f"[DEBUG] Starting scenario generation with sim_time={sim_time}, min_difference_time={min_difference_time}")
        #     print(f"[DEBUG] Original 'a': {a}")
        #     print(f"[DEBUG] Original 'b': {b}")
        #     print(f"[DEBUG] Recipe Cluster: {list(problem_df['Recipe_Cluster'])}")

        # Step 1: Find the next unique `a` value greater than `sim_time`
        future_a_values = a[1:][a[1:] > sim_time]
        if len(future_a_values) == 0:
            if self.debug:
                print("[DEBUG] No jobs found after the current sim_time.")
            return {
                'a': a,
                'b': b,
                'isDeterministic': True,
                'Scenarios': {}
            }

        next_a_value = np.min(future_a_values)
        next_jobs_mask = (a[1:] == next_a_value)
        next_jobs_df = problem_df[next_jobs_mask]
        next_recipe_clusters = np.sort(next_jobs_df['Recipe_Cluster'].unique())

        # Use numpy for probability calculations
        cluster_labels, cluster_counts = np.unique(next_jobs_df['Recipe_Cluster'], return_counts=True)
        total_next_jobs = cluster_counts.sum()
        next_probabilities = cluster_counts / total_next_jobs

        # Create dict for Scenarios
        scenarios = {label: prob for label, prob in zip(cluster_labels, next_probabilities)}

        # if self.debug:
        #     print(f"[DEBUG] Next jobs mask: {next_jobs_mask}")
        #     print(f"[DEBUG] Next unique `a` value: {next_a_value}")
        #     print(f"[DEBUG] Cluster labels: {cluster_labels}")
        #     print(f"[DEBUG] Cluster counts: {cluster_counts}")
        #     print(f"[DEBUG] Next probabilities: {next_probabilities}")
        #     print(f"[DEBUG] Scenarios: {scenarios}")

        # Step 2: Check if the scenario is deterministic
        if len(next_recipe_clusters) == 1 or abs(sim_time - next_a_value) <= min_difference_time:
            if self.debug:
                print(f"[DEBUG] Deterministic scenario identified.")
            return {
                'a': a,
                'b': b,
                'isDeterministic': True,
                'Scenarios': scenarios
            }

        # Step 3: Process for stochastic scenarios
        processing_minutes = problem_df[['Machine1_ProcessMinutes', 'Machine2_ProcessMinutes']].to_numpy(dtype=float)

        # Handle bigM values based on nanmean assumption
        if nanmean:
            avg_processing_times = np.nanmean(# Temporarily treat bigM=10000 as np.nan during computation
                np.where(processing_minutes == bigM, np.nan, processing_minutes),
                axis=1)
        else:
            # Treat np.nan as 10000 to handle the case where bigM was interpreted as np.nan
            avg_processing_times = np.mean(
                np.where(np.isnan(processing_minutes), bigM, processing_minutes),
                axis=1)


        # Find the index of the max processing time for each relevant recipe cluster
        cluster_max_indices = (
            next_jobs_df.groupby('Recipe_Cluster')
            .apply(lambda df: df.index[np.argmax(avg_processing_times[df.index])], include_groups=False)
            .to_numpy()
        )

        selected_jobs = problem_df.loc[cluster_max_indices]

        # if self.debug:
        #     print(f"[DEBUG] Relevant jobs for stochastic scenarios:")
        #     display(next_jobs_df)
        #     print(f"[DEBUG] Selected jobs for stochastic scenarios:")
        #     display(selected_jobs)

        # Step 4: Create scenarios using numpy operations
        selected_indices = selected_jobs.index.to_numpy() + 1  # Adjust for prepended 0 in a and b
        num_scenarios = len(selected_indices)

        a_scenarios = np.tile(a, (num_scenarios, 1))
        b_scenarios = np.tile(b, (num_scenarios, 1))

        adjusted_a_values = np.maximum(
            a_scenarios[np.arange(num_scenarios), selected_indices] - earliness,
            sim_time + min_difference_time
        )
        # Ensure b mirrors the same adjustment applied to a
        adjusted_b_values = b_scenarios[np.arange(num_scenarios), selected_indices] - (
            a_scenarios[np.arange(num_scenarios), selected_indices] - adjusted_a_values
        )

        # Apply the adjustments
        a_scenarios[np.arange(num_scenarios), selected_indices] = adjusted_a_values
        b_scenarios[np.arange(num_scenarios), selected_indices] = adjusted_b_values

        # if self.debug:
        #     print(f"[DEBUG] Final Scenarios for 'a':\n{a_scenarios}")
        #     print(f"[DEBUG] Final Scenarios for 'b':\n{b_scenarios}")

        return {
            'a': a_scenarios,
            'b': b_scenarios,
            'isDeterministic': False,
            'Scenarios': scenarios
        }
        
    def stochastic_iterative_solving(self):
        """
        Iteratively solves the stochastic version of the VRP using a similar approach as iterative_solving.
        """
        time_window_delta = self.delta_start  # Start delta
        SIMTIME = np.unique(self.a_true)  # Unique SIMTIME values
        
        # Initialize solution trackers
        prev_t_ik = {}
        prev_x_ijk = {}
        
        cumulative_x_fixes = set()  # Track cumulative fixings
        
        for sim_time in SIMTIME:
            if self.debug: print(f"[DEBUG] Simulating for SIMTIME = {sim_time}")
    
            # Update simulated a and b based on SIMTIME
            mask = sim_time >= self.a_true
            self.a_sim = np.where(mask, self.a_true, self.a_sim)
            self.b_sim = np.where(mask, self.b_true, self.b_sim)

            result = self.scenario_generating(sim_time)
            a_scenarios, b_scenarios, isDeterministic, scenarios = (result['a'], 
                                                                    result['b'], 
                                                                    result['isDeterministic'], 
                                                                    result['Scenarios'])
            

            # Create and update model
            if isDeterministic:
                self.create_model(time_window_delta)
            else:
                self.create_model_stochastic(time_window_delta, a_scenarios, b_scenarios, scenarios)
            
            # Apply cumulative constraints
            if self.debug:
                print("[DEBUG] Applying cumulative constraints from previous iterations:")
            for (i, j, k) in cumulative_x_fixes:
                self.add_cumulative_constraints(i, j, k)
            
            # Apply new fixings based on t_ik and SIMTIME
            if self.debug:
                print("[DEBUG] Applying new fixings based on t_ik values and SIMTIME:")
            for (i, k), t_value in prev_t_ik.items():
                if sim_time < t_value:
                    continue  # Skip if SIMTIME not reached
        
                x_values = prev_x_ijk.get((i, k), {})
                for j, x_value in x_values.items():
                    if x_value != 1:  # Check for exact equality with 1
                        if self.debug: 
                            print(f"[DEBUG] Skipping x[{i},{j},{k}] as value {x_value} is not close enough to 1.")
                        continue  # Only fix values close to 1
    
                    # Apply cumulative constraint and track
                    self.add_cumulative_constraints(i, j, k)
                    cumulative_x_fixes.add((i, j, k))
            
            # Solve model               
            self.solve()
            
            # Retrieve and print solution
            if isDeterministic:
                solution = self.get_solution()
            else:
                solution = self.get_solution_stochastic()
            
            if self.result is None:
                if self.debug:
                    print("[ERROR] Solver returned no result. Aborting iterations.")
                break  # Exit loop if no result

            if self.debug:
                print(f"x_assignments: {solution['x_variables']}")
                print(f"t_assignments: {solution['t_variables']}")
    
            # Store solution if feasible or optimal
            if self.result.solver.termination_condition in [
                pyo.TerminationCondition.optimal,
                pyo.TerminationCondition.feasible, 
                pyo.TerminationCondition.maxTimeLimit,
            ]:
                
                if isDeterministic:
                    if self.debug:
                        print("[DEBUG] Extracting t_ik values for deterministic model.")
                    prev_t_ik = {
                        (i, k): value
                        for (i, k), value in self.model.t_ik.get_values().items()
                    }
                else:
                    if self.debug:
                        print("[DEBUG] Extracting t_ikw values for stochastic model and taking max across scenarios.")
                    t_ikw_values = self.model.t_ikw.get_values()
                    prev_t_ik = {}
                    for (i, k, w), value in t_ikw_values.items():
                        if (i, k) not in prev_t_ik:
                            prev_t_ik[(i, k)] = int(round(value))
                        else:
                            prev_t_ik[(i, k)] = int(round(max(prev_t_ik[(i, k)], value)))

                if self.debug: print(prev_t_ik)
                # Track x values close to 1
                prev_x_ijk = {}
                for (i, j, k), value in self.model.x.get_values().items():
                    if value == 1:  # Check for exact equality with 1
                        if (i, k) not in prev_x_ijk:
                            prev_x_ijk[(i, k)] = {}  # Initialize inner dict
                        prev_x_ijk[(i, k)][j] = value  # Store j and value in inner dict
                
                # Update warm_start with the current x values
                self.warm_start = np.array([
                    [[round(self.model.x[i, j, k + 1].value) for k in range(self.warm_start.shape[2])]
                     for j in range(self.warm_start.shape[1])]
                    for i in range(self.warm_start.shape[0])
                ], dtype=int)

                if self.debug:
                    print("[DEBUG] Solution saved for next warm start.")
            else:
                if self.debug:
                    print("[DEBUG] No feasible solution found.")
                break  # Exit loop if infeasible

    def get_solution_stochastic(self, rounding_tolerance=8):
        if self.result is None or self.result.solver.termination_condition not in [
            pyo.TerminationCondition.optimal,
            pyo.TerminationCondition.maxTimeLimit,
        ]:
            return {
                "status": "Aborted",
                "objective_value": None,
                "solution_time": self.solution_time,
                "makespan": None,
                "x_variables": {},
                "t_variables": {},
                "decision_matrix": None
            }
    
        num_jobs = len(self.model.N)
        num_machines = len(self.model.k)
        decision_matrix = np.zeros((num_jobs, num_jobs, num_machines),
                                   dtype=int)
    
        solution = {
            "status": self.result.solver.termination_condition,
            "objective_value": round(pyo.value(self.model.objective), 2),
            "solution_time": self.solution_time,
            "makespan": {}, 
            "x_variables": {},
            "t_variables": {},  # Updated to store `t_ikw` values
            "decision_matrix": decision_matrix
        }
    
        # Populate and sort t_ikw variables
        sorted_t_ikw_values = sorted(
            (
                (i, k, w, int(round(value)))
                for (i, k, w), value in self.model.t_ikw.get_values().items()
                if round(value, rounding_tolerance) >= 1
            ),
            key=lambda x: (x[2], x[1], x[3])  # Sort by scenario w, then machine k, then time value
        )
        
        # Store sorted t_ikw values
        for i, k, w, value in sorted_t_ikw_values:
            solution["t_variables"][f"t_ikw[{i},{k},{w}]"] = int(round(value, rounding_tolerance))
    
        # Collect and sort x variables
        sorted_x_values = sorted(
            ((i, j, k, int(round(value))) for (i, j, k), value in self.model.x.get_values().items() if round(value) >= 1),
            key=lambda x: (x[2], x[0], x[1])  # Sort by k, then i, then j
        )
        
        # Store x values in matrix
        for i, j, k, value in sorted_x_values:
            decision_matrix[i, j, k - 1] = value
            solution["x_variables"][f"x[{i},{j},{k}]"] = int(round(value))
    
        # Return processed solution
        return solution


##### Debugging specific Problem Numbers

In [115]:
# Preparing the data into matrices and whatnot
QueueSize, ProblemNumber = 25, 277

#QueueSize: 25; ProblemNumber: 277

print(f'QueueSize:{QueueSize}; ProblemNumber:{ProblemNumber}')

# Load problem df
#file_path = f'ProblemSets/7Queue_debugging_development'
file_path = f'ProblemSets/Training_Queue{QueueSize}'
problem_df = pd.read_pickle(f'{file_path}/Problem_{ProblemNumber}_jobs.pkl')

# Load cost matrix
cost_matrix = np.load(f'{file_path}/Problem_{ProblemNumber}_cost_matrix.npy')

# Load warm start solution from AI_LPT DES simulation to start!
## Finds the  best EXPECTED SolutionNumber to use
SolutionNumber_WarmStart = get_best_AI_LPT(QueueSize, ProblemNumber)['SolutionNumber'].iloc[0]
solution_dict = pd.read_pickle(f'{file_path}/Solutions/AI_LPT/Problem_{ProblemNumber}_Solution_{SolutionNumber_WarmStart}.pkl') # more efficient to store a dict of the solutions
warm_start_obj_value,  warm_start_transition_matrix = solution_dict['obj_value'], np.array(solution_dict['transition_matrix']) # and then grab all the elements needed from the pkl dict
print(f"Providing MIP of {warm_start_obj_value}")

# Initialize VRPSolver
# Hyperparameters
delta_start_map = { # queuesize to delta start
    10: 650,
    25: 650,
    50: 2000,
    75: 3000
}
delta_start = delta_start_map.get(QueueSize, 1000) # between 0 and 3000 minutes, integers
runtime_seconds_max = 10 # between 0.25 and 30 seconds (assumed much less than 5 minutes for speed and simulating)
num_threads = 1


solver = StochasticVRPSolver(
    cost_matrix=cost_matrix,
    problem_df=problem_df,
    problemset_num=QueueSize,
    problem_num=ProblemNumber,
    warm_start=warm_start_transition_matrix,
    debug=False,
    stochastic_parameters={
        "nanmean": True  # How to average multiple machine processing times with bigM/NaNs... ignore or bake in?
        #"min_difference_time": 5,
        #"earliness": 30,
    },
    hyperparams={
         "delta_start": delta_start,  # Between 0 and 3000 minutes
         "num_threads": num_threads,  # Adjusted based on available cores
         "time_limit": runtime_seconds_max,  # Solver time limit in seconds
         "symmetry_breaking": False,  # Enable symmetry-breaking constraints
         "mip_focus": 2,  # MIP focus control # MIP focus: 0=auto, 1=optimality, 2=feasibility, 3=balance
         "cuts": 3,  # Cut generation control # Cut generation: 0=auto, 1=minimal, 2=moderate, 3=aggressive
         "heuristics":  0.7218608689267029,  # Fraction of time for primal heuristics  # Fraction of time for primal heuristics (0.0 to 1.0)
         "presolve": 1  # Presolve phase # Presolve: -1=auto, 0=off, 1=conservative, 2=aggressive
    }
)
# {'delta_start': 1685, 'symmetry_breaking': True, 'mip_focus': 2, 'cuts': 3, 'heuristics': 0.7218608689267029, 'presolve': 1}

# Run optimization with warm start
solver.stochastic_iterative_solving()


# Get the final solution after the iteration process
solution = solver.get_solution()

# Output solution
solution

QueueSize:25; ProblemNumber:277
Providing MIP of 828.5


{'status': <TerminationCondition.optimal: 'optimal'>,
 'objective_value': 585.25,
 'solution_time': 10.19,
 'makespan': 648,
 'x_variables': {'x[0,1,1]': 1,
  'x[1,13,1]': 1,
  'x[6,24,1]': 1,
  'x[13,15,1]': 1,
  'x[14,18,1]': 1,
  'x[15,14,1]': 1,
  'x[16,6,1]': 1,
  'x[17,23,1]': 1,
  'x[18,17,1]': 1,
  'x[20,22,1]': 1,
  'x[22,16,1]': 1,
  'x[23,20,1]': 1,
  'x[24,0,1]': 1,
  'x[0,2,2]': 1,
  'x[2,7,2]': 1,
  'x[3,8,2]': 1,
  'x[4,21,2]': 1,
  'x[5,4,2]': 1,
  'x[7,3,2]': 1,
  'x[8,12,2]': 1,
  'x[9,10,2]': 1,
  'x[10,25,2]': 1,
  'x[11,0,2]': 1,
  'x[12,5,2]': 1,
  'x[19,9,2]': 1,
  'x[21,19,2]': 1,
  'x[25,11,2]': 1},
 't_variables': {'t_ik[13,1]': 150,
  't_ik[15,1]': 196,
  't_ik[14,1]': 240,
  't_ik[18,1]': 305,
  't_ik[17,1]': 333,
  't_ik[23,1]': 350,
  't_ik[20,1]': 507,
  't_ik[22,1]': 526,
  't_ik[16,1]': 555,
  't_ik[6,1]': 624,
  't_ik[24,1]': 632,
  't_ik[26,1]': 648,
  't_ik[7,2]': 70,
  't_ik[3,2]': 167,
  't_ik[8,2]': 175,
  't_ik[12,2]': 225,
  't_ik[5,2]': 293,
  

### Hyperparameter Tuning

#### Saving, loading, and running Optuna studies so that the work isn't lost

In [3]:
import optuna
import os
import pickle



def save_study(study, folder_path="optuna_studies", study_name="default_study"):
    # Ensure the folder exists
    os.makedirs(folder_path, exist_ok=True)
    
    # Save the study as a pickle file
    file_path = os.path.join(folder_path, f"{study_name}.pkl")
    with open(file_path, "wb") as f:
        pickle.dump(study, f)
    print(f"Study saved to {file_path}")


def load_study(folder_path="optuna_studies", study_name="default_study"):
    # Load the study from a pickle file
    file_path = os.path.join(folder_path, f"{study_name}.pkl")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"No study found at {file_path}")
    
    with open(file_path, "rb") as f:
        study = pickle.load(f)
    print(f"Study loaded from {file_path}")

    # Determine the number of completed trials
    completed_trials = len(study.get_trials(deepcopy=False, 
                                            states=[optuna.trial.TrialState.COMPLETE]))
    print(f"[INFO] Starting from Trial {completed_trials}")
    
    return study


/home/jblitz/miniconda3/envs/MathOpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import time
import traceback

def run_combined_AFK_cycle(
    study_func,
    queue_sizes,
    n_trials=100,
    n_jobs=1,
    folder="optuna_studies",
    max_retries=3,
    retry_delay=60,
    run_delay=300,
    **kwargs,
):
    """
    Combine cycle and retry logic for running studies across multiple queue sizes.

    Args:
        study_func (callable): The study function to run.
        queue_sizes (list): List of queue sizes to iterate over.
        n_trials (int): Number of trials per run.
        n_jobs (int): Number of parallel jobs.
        folder (str): Folder to save study results.
        max_retries (int): Maximum retries before stopping a study for a queue size.
        retry_delay (int): Seconds to wait after a failure.
        run_delay (int): Seconds to wait between successful runs.
        **kwargs: Additional arguments for the study function.
    """
    while True:  # Continuous loop to cycle through queue sizes
        for queue_size in queue_sizes:
            study_name = f"Queue{queue_size}_Study"
            retries = 0

            while retries < max_retries:
                try:
                    # Run the study function
                    study_func(queue_size=queue_size, n_trials=n_trials, n_jobs=n_jobs, study_name=study_name, folder_path=folder, **kwargs)                    
                    # Wait between successful runs
                    print(f"Sleeping zZz  zZz  Zzz")
                    time.sleep(run_delay)
                    break  # Exit retry loop on success

                except Exception as e:
                    retries += 1
                    print(f"[ERROR] Error during '{study_name}' (Retry {retries}/{max_retries}): {e}")
                    traceback.print_exc()
                    time.sleep(retry_delay)

                except KeyboardInterrupt:
                    print("[INFO] Interrupted by user. Exiting.")
                    return  # Exit all loops on user interruption

                except BaseException as e:
                    print(f"[CRITICAL] Unexpected error: {e}. Stopping.")
                    return  # Exit all loops on critical error

            else:
                print(f"[ERROR] Max retries ({max_retries}) reached for '{study_name}'. Moving to the next queue size.")



### Hyperparameter Tuning for Deterministic Optimization

In [8]:
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import multiprocessing
import tempfile
import shutil
import os
import optuna
from optuna.samplers import TPESampler, RandomSampler
import warnings
warnings.filterwarnings("ignore", category=optuna.exceptions.ExperimentalWarning)
optuna.logging.set_verbosity(optuna.logging.INFO)


# Define QueueSize-specific problem number ranges
QUEUE_PROBLEM_RANGE = {
    10: (1, 750),
    25: (1, 300),
    #50: (1, 150), # obsoleted
    75: (1, 100),
}

# Define `minimum delta_start` based on observed max values (Figures in Chapter IV)
delta_start_min_by_queue_size = {
    25: 646,  # for queue size 25
    75: 2495  # for queue size 75
}

def generate_problem(queue_size):
    problem_range = QUEUE_PROBLEM_RANGE.get(queue_size, (1, 750))
    problem_number = np.random.randint(*problem_range)
    return queue_size, problem_number

def load_problem_data(queue_size, problem_number, training=True):
    """
    Load problem data for a given QueueSize and ProblemNumber.

    Args:
        queue_size (int): The size of the queue (e.g., 25 or 75).
        problem_number (int): The problem number to load.
        training (bool): Whether to load from the training folder prefix. 
                         Defaults to True. If False, loads from non-training folder prefix.

    Returns:
        tuple: problem_df, cost_matrix, warm_start_transition_matrix, traditional_SPT_obj_value
    """
    # Determine folder prefix based on training flag
    folder_prefix = "Training_" if training else ""
    file_path = f"ProblemSets/{folder_prefix}Queue{queue_size}"
    print(f'file_path used:{file_path}')

    # Load traditional SPT objective value
    traditional_SPT_obj_value = pd.read_pickle(
        f"{file_path}/Solutions/Traditional_true_SPT/Problem_{problem_number}_Solution_1.pkl"
    )['obj_value'] # now using the true Traditional_SPT for more accurate scoring
    
    # Load problem DataFrame and cost matrix
    problem_df = pd.read_pickle(f"{file_path}/Problem_{problem_number}_jobs.pkl")
    cost_matrix = np.load(f"{file_path}/Problem_{problem_number}_cost_matrix.npy")
    
    # Load warm start solution
    SolutionNumber_WarmStart = get_best_AI_LPT(queue_size, problem_number)['SolutionNumber'].iloc[0]
    solution_dict = pd.read_pickle(
        f"{file_path}/Solutions/AI_LPT/Problem_{problem_number}_Solution_{SolutionNumber_WarmStart}.pkl"
    )
    warm_start_obj_value = solution_dict['obj_value']
    warm_start_transition_matrix = np.array(solution_dict['transition_matrix'])
    
    return problem_df, cost_matrix, warm_start_transition_matrix, traditional_SPT_obj_value


def optuna_objective(queue_size, max_time_seconds=10):
    delta_start_min = delta_start_min_by_queue_size.get(queue_size, 0)  # Default to 0 if queue_size not found
    def objective(trial):
        # Sample hyperparameters
        delta_start = trial.suggest_int("delta_start", delta_start_min, 3000)
        runtime_seconds_max = max_time_seconds
        symmetry_breaking = trial.suggest_categorical("symmetry_breaking", [True, False])
        mip_focus = trial.suggest_int("mip_focus", 0, 3)  # Add MIP focus
        cuts = trial.suggest_int("cuts", 0, 3)  # Add cut generation
        heuristics = trial.suggest_float("heuristics", 0.0, 1.0)  # Add heuristics control
        presolve = trial.suggest_int("presolve", -1, 2)  # Add presolve control
        #nodefile_start = trial.suggest_float("nodefile_start", 0.1, 1.0)  # Add node file threshold (0.1 GB to 1.0 GB) # Doesn't affect the optimization too much

        # Generate problem
        _, problem_number = generate_problem(queue_size)
        print(f"\rQueueSize: {queue_size}; ProblemNumber: {problem_number}")
        problem_df, cost_matrix, warm_start_transition_matrix, traditional_SPT_obj_value = load_problem_data(queue_size, problem_number) ## traditional_SPT should be expected for training
        
        # Setup solver
        solver = VRPSolver(
            cost_matrix=cost_matrix,
            problem_df=problem_df,
            problemset_num=queue_size,
            problem_num=problem_number,
            warm_start=warm_start_transition_matrix,
            debug=False,
            hyperparams={
                "delta_start": delta_start,
                "time_limit": runtime_seconds_max,
                "symmetry_breaking": symmetry_breaking,
                "mip_focus": mip_focus,
                "cuts": cuts,
                "heuristics": heuristics,
                "presolve": presolve,
                #"nodefile_start": nodefile_start,
                "num_threads": 23,
            }
        )

        # Solve problem
        solver.iterative_solving() # deterministic only
        solution = solver.get_solution()

        # Scoring
        optimization_algorithm_obj_value = solution["objective_value"]
        
        if optimization_algorithm_obj_value  is None:
            return -100 # infeasible means worst-case, thus penalizing
        
        O, T = optimization_algorithm_obj_value, traditional_SPT_obj_value 
        
        improvement_relative_percent = (T - O) / max(T, 1e-4) * 100 # no 0's allowed for divide by zero

        return improvement_relative_percent

    return objective

def optuna_objective_stochastic(queue_size, max_time_seconds=10):
    delta_start_min = delta_start_min_by_queue_size.get(queue_size, 0)  # Default to 0 if queue_size not found
    def objective(trial):
        # Sample stochastic hyperparameters
        delta_start = trial.suggest_int("delta_start", delta_start_min, 3000)
        symmetry_breaking = False # (informed by prior)
        mip_focus = trial.suggest_int("mip_focus", 0, 3)
        cuts = trial.suggest_int("cuts", 2, 3)  # Add cut generation (informed by prior)
        heuristics = trial.suggest_float("heuristics", 0.5, 1.0) # (informed by prior)
        presolve = trial.suggest_int("presolve", 0, 2) # (informed by prior)
        nanmean = trial.suggest_categorical("nanmean", [True, False])  # New stochastic parameter

        # Generate problem
        _, problem_number = generate_problem(queue_size)
        print(f"\rQueueSize: {queue_size}; ProblemNumber: {problem_number}")
        problem_df, cost_matrix, warm_start_transition_matrix, traditional_SPT_obj_value = load_problem_data(queue_size, problem_number)

        # Setup solver
        solver = StochasticVRPSolver(
            cost_matrix=cost_matrix,
            problem_df=problem_df,
            problemset_num=queue_size,
            problem_num=problem_number,
            warm_start=warm_start_transition_matrix,
            debug=False,
            stochastic_parameters={
                "nanmean": nanmean,  # Use the sampled `nanmean` parameter
            },
            hyperparams={
                "delta_start": delta_start,
                "time_limit": max_time_seconds,
                "symmetry_breaking": symmetry_breaking,
                "mip_focus": mip_focus,
                "cuts": cuts, 
                "heuristics": heuristics,
                "presolve": presolve,
                "num_threads": 23,
            }
        )

        # Solve problem
        solver.stochastic_iterative_solving() # allows stochastic or deterministic based on iteration and next jobs' scenarios
        solution = solver.get_solution()

        # Scoring
        optimization_algorithm_obj_value = solution["objective_value"]

        if optimization_algorithm_obj_value is None:
            return -100  # Penalize infeasible solutions

        O, T = optimization_algorithm_obj_value, traditional_SPT_obj_value
        improvement_relative_percent = (T - O) / max(T, 1e-4) * 100  # Avoid divide by zero
        return improvement_relative_percent

    return objective

def parallel_study(
    queue_size,
    n_trials,
    n_jobs=1,
    study_name="default_study",
    folder_path="optuna_studies",
    pruner=None,
):
    """
    Run a parallel Optuna study, saving results to a folder.

    Args:
        queue_size (int): Queue size for study.
        n_trials (int): Number of trials.
        n_jobs (int): Number of parallel jobs.
        study_name (str): Name of study.
        folder_path (str): Folder to save study results.
        pruner (optuna.pruners.BasePruner, optional): Pruner to use for study.
    """
    if pruner is None:
        pruner = None #optuna.pruners.MedianPruner(n_startup_trials=100) # obsoleted, do not need given solver nature as self-contained

    # Load or create a study
    try:
        study = load_study(folder_path, study_name)
    except FileNotFoundError:
        # Use RandomSampler for the first trial
        random_sampler = RandomSampler()
        study = optuna.create_study(
            study_name=study_name, 
            direction="maximize", 
            pruner=pruner, 
            sampler=random_sampler
        )

    os.environ["OPTUNA_SHOW_PROGRESS_BAR"] = "1" ## Progress bar

    # Run the first trial with RandomSampler
    study.optimize(
        lambda trial: optuna_objective(queue_size)(trial),
        n_trials=1,  # Only one trial with RandomSampler
        n_jobs=1,
    )

    # Switch to TPESampler for subsequent trials
    tpe_sampler = TPESampler(multivariate = True, group = True, warn_independent_sampling=False) # Most important for bayesian optimization and sampling to be dependent paramaters with groupings (turned off a warning because mid-study I changed the lowerbound for delta_start)
    study.sampler = tpe_sampler  # Update sampler for subsequent trials

    os.environ["OPTUNA_SHOW_PROGRESS_BAR"] = "1" ## Progress bar

    # Optimize remaining trials
    study.optimize(
        lambda trial: optuna_objective(queue_size)(trial),
        n_trials=n_trials - 1,  # Subtract the first trial
        n_jobs=n_jobs,
    )

    # Save the study after optimization
    save_study(study, folder_path, study_name)

def parallel_study_stochastic(
    queue_size,
    n_trials,
    n_jobs=1,
    study_name="stochastic_study",
    folder_path="optuna_studies_stochastic",
    pruner=None,
):
    if pruner is None:
        pruner = None  # No pruner needed for this study

    # Load or create a study
    try:
        study = load_study(folder_path, study_name)
    except FileNotFoundError:
        study = optuna.create_study(
            study_name=study_name,
            direction="maximize",
            pruner=pruner,
            sampler=RandomSampler(),  # Use RandomSampler for the first trial
        )

    # Run the first trial with RandomSampler
    study.optimize(
        lambda trial: optuna_objective_stochastic(queue_size)(trial),
        n_trials=1,
        n_jobs=1,
    )

    # Switch to TPESampler for subsequent trials
    study.sampler = TPESampler(multivariate=True, group=True, warn_independent_sampling=False)
    study.optimize(
        lambda trial: optuna_objective_stochastic(queue_size)(trial),
        n_trials=n_trials - 1,  # Subtract the first trial
        n_jobs=n_jobs,
    )

    # Save the study
    save_study(study, folder_path, study_name)


/home/jblitz/miniconda3/envs/MathOpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
## 25 then 75 in a cycle

In [ ]:
# deterministic hyperparameter search
if __name__ == "__main__":
    run_combined_AFK_cycle(
        study_func=parallel_study,  # Replace with your actual study function
        queue_sizes=[25, 25, 75],   # List of QueueSizes to cycle through (balancing out the 25's to be more than 75)
        n_trials=2,                 # Number of trials per run
        folder="optuna_studies_deterministic",    # Folder to save study results
        retry_delay=7,             # Wait 7 seconds on failure
        run_delay=0.25                 # Wait 0.25 seconds between successful runs
    )

Study loaded from optuna_studies_deterministic/Queue25_Study.pkl
[INFO] Starting from Trial 1081
QueueSize: 25; ProblemNumber: 276


[I 2024-11-18 22:05:48,915] Trial 1081 finished with value: 56.74149167270094 and parameters: {'delta_start': 1180, 'symmetry_breaking': False, 'mip_focus': 2, 'cuts': 1, 'heuristics': 0.008249707834533385, 'presolve': 0}. Best is trial 854 with value: 62.73170731707317.


QueueSize: 25; ProblemNumber: 280


[I 2024-11-18 22:07:42,458] Trial 1082 finished with value: 15.406976744186046 and parameters: {'delta_start': 1058, 'symmetry_breaking': False, 'mip_focus': 2, 'cuts': 2, 'heuristics': 0.9909578780738033, 'presolve': 1}. Best is trial 854 with value: 62.73170731707317.


Study saved to optuna_studies_deterministic/Queue25_Study.pkl
Sleeping zZz  zZz  Zzz
Study loaded from optuna_studies_deterministic/Queue25_Study.pkl
[INFO] Starting from Trial 1083
QueueSize: 25; ProblemNumber: 199


...

QueueSize: 25; ProblemNumber: 200


[I 2024-11-19 10:30:47,328] Trial 1324 finished with value: 15.528478322470955 and parameters: {'delta_start': 1027, 'symmetry_breaking': False, 'mip_focus': 2, 'cuts': 0, 'heuristics': 0.04060277636308339, 'presolve': 0}. Best is trial 854 with value: 62.73170731707317.


Study saved to optuna_studies_deterministic/Queue25_Study.pkl
Sleeping zZz  zZz  Zzz
Study loaded from optuna_studies_deterministic/Queue75_Study.pkl
[INFO] Starting from Trial 1131
QueueSize: 75; ProblemNumber: 26


[I 2024-11-19 10:34:01,031] Trial 1131 finished with value: 17.120731479753683 and parameters: {'delta_start': 2703, 'symmetry_breaking': False, 'mip_focus': 3, 'cuts': 2, 'heuristics': 0.5942894615736378, 'presolve': 1}. Best is trial 908 with value: 23.248006714225767.


QueueSize: 75; ProblemNumber: 60


#### When choosing `Delta_Start`, notes for Validations and Future Optuna studies

- Use the visualization from Main when choosing the validation methodology to avoid wasting time on optimization failures.
- When choosing `Delta_Start` for stochastic optimization, change the lowest possible values to reduce the search space and require fewer iterations. It will still fail when more scenarios are involved and there is more variability; however, for robustness, it will also be faster and less of a waste of time for the studies!~

#### Validating which parameters are the most robust

Choose based on the best averages for improvements, ignoring the fails, the idea is that the fails can be easily ignored, even though it is desirable to avoid them in the first place which is what we did.

The parameters with the best results across 10 consistent jobs sampled per ProblemSet, will be used.

In [4]:
# Problem ranges for queue sizes
QUEUE_PROBLEM_RANGE = {
    25: (1, 300),
    75: (1, 100)
}

# Dictionary to store validation samples
validation_samples = {}

# Random seed for reproducibility
np.random.seed(42)

# Sampling percentage
sample_size = 10

# Generate a shared sample set for both deterministic and stochastic
for queue_size, (start, end) in QUEUE_PROBLEM_RANGE.items():
    problem_numbers = np.arange(start, end + 1)
    np.random.shuffle(problem_numbers) # shuffles the order
    validation_samples[queue_size] = problem_numbers[:sample_size]  # draws from the deck of ProblemNumbers per QueueSize


In [5]:
validation_samples

{25: array([204, 267, 153,  10, 234, 227, 197, 110,   6, 176]),
 75: array([ 3, 34, 68, 97, 51, 84, 89, 77, 66, 41])}

In [76]:
# ### (Do not run again, already ran)

# import pandas as pd
# import pickle
# import os

# # Define minimum delta_start for each queue size
# delta_start_min_by_queue_size = {
#     25: 646,
#     75: 2495
# }

# # Load studies function
# def load_study(folder, name):
#     with open(os.path.join(folder, f"{name}.pkl"), "rb") as f:
#         return pickle.load(f)

# # Load studies
# folder_path = "optuna_studies_deterministic"
# study_names = ["Queue25_Study", "Queue75_Study"]
# studies = {name: load_study(folder_path, name) for name in study_names}

# # Extract trial data into a Pandas DataFrame
# all_trials_data = []
# for study_name, study in studies.items():
#     queue_size = int(study_name.split("Queue")[1].split("_")[0])
#     delta_start_min = delta_start_min_by_queue_size.get(queue_size, 0)

#     for trial in study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE]):
#         # Exclude 'delta_start' from the hyperparameters
#         hyperparameters = {k: v for k, v in trial.params.items() if k != "delta_start"}
#         all_trials_data.append({
#             "StudyName": study_name,
#             "QueueSize": queue_size,
#             "TrialNumber": trial.number,
#             "DeltaStart": trial.params.get("delta_start", 0),
#             "ObjectiveValue": trial.values[0] if trial.values else None,
#             "Hyperparameters": hyperparameters  # Store hyperparameters as a dict
#         })

# # Convert to DataFrame
# trials_df = pd.DataFrame(all_trials_data)

# # Filter by delta_start_min and valid ObjectiveValue
# filtered_trials_df = trials_df[
#     (trials_df["DeltaStart"] > trials_df["QueueSize"].map(delta_start_min_by_queue_size))
# ]

# # Sort and group by queue size
# sorted_trials_df = (
#     filtered_trials_df.sort_values(by=["QueueSize", "ObjectiveValue"], ascending=[True, False])
# )

# # Select top 3 rows per StudyName (Deterministic)
# print(f"Table for Top 3 Best Params for Deterministic Optimization Validation")
# top_3_per_study = sorted_trials_df.groupby("StudyName").head(3).copy()
# min_delta_start_per_queue = top_3_per_study.groupby("QueueSize")["DeltaStart"].transform("min") # per row
# top_3_per_study["DeltaStart_adjusted"] = min_delta_start_per_queue
# top_3_per_study["Hyperparameters"] = top_3_per_study.apply(
#     lambda row: {**row["Hyperparameters"], "delta_start": row["DeltaStart_adjusted"]}, axis=1
# )

# display(top_3_per_study)


Table for Top 3 Best Params for Deterministic Optimization Validation


,StudyName,QueueSize,TrialNumber,DeltaStart,ObjectiveValue,Hyperparameters,DeltaStart_adjusted
854,Queue25_Study,25,854,2369,62.731707,"{'symmetry_breaking': False, 'mip_focus': 1, '...",1291
973,Queue25_Study,25,973,2705,62.731707,"{'symmetry_breaking': False, 'mip_focus': 2, '...",1291
698,Queue25_Study,25,698,1291,62.390244,"{'symmetry_breaking': True, 'mip_focus': 2, 'c...",1291
2121,Queue75_Study,75,796,2509,22.071674,"{'symmetry_breaking': False, 'mip_focus': 1, '...",2509
2334,Queue75_Study,75,1009,2725,21.955518,"{'symmetry_breaking': True, 'mip_focus': 0, 'c...",2509
2046,Queue75_Study,75,721,2875,21.930340,"{'symmetry_breaking': False, 'mip_focus': 1, '...",2509


{25: array([204, 267, 153,  10, 234, 227, 197, 110,   6, 176]),
 75: array([ 3, 34, 68, 97, 51, 84, 89, 77, 66, 41])}

#### Creating a Run Plan Sequence for Future away from keyboard (AFK) functions

This was very time-consuming, so having an AFK function was crucial. It needed to be fault-tolerant and save and load from a pickle file.

In [87]:
# do not run again; already ran

# import pandas as pd
# import itertools
# import pickle

# # Generate run plan data
# run_plan_data = []

# # Iterate over queue sizes and problems
# for queue_size, problems in validation_samples.items():
#     # Filter top hyperparameters
#     top_3_hyperparams = top_3_per_study[top_3_per_study["QueueSize"] == queue_size]["Hyperparameters"]
    
#     # Combine problems and hyperparameters
#     for problem_number, hyperparams in itertools.product(problems, top_3_hyperparams):
#         run_plan_data.append({
#             "QueueSize": queue_size,  # Add queue size
#             "ProblemNumber": problem_number,  # Add problem number
#             "Hyperparameters": hyperparams,  # Add hyperparameter set
#             "ObjectiveValue": None  # Placeholder for results
#         })

# # Convert to DataFrame
# run_plan_df = pd.DataFrame(run_plan_data)

# # Save run plan
# run_plan_file = "validation_run_plan_deterministic.pkl"

# # with open(run_plan_file, "wb") as f:
# #     pickle.dump(run_plan_df, f)

# # Display the run plan
# display(run_plan_df)


,QueueSize,ProblemNumber,Hyperparameters,ObjectiveValue
0,25,204,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
1,25,204,"{'symmetry_breaking': False, 'mip_focus': 2, '...",None
2,25,204,"{'symmetry_breaking': True, 'mip_focus': 2, 'c...",None
3,25,267,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
4,25,267,"{'symmetry_breaking': False, 'mip_focus': 2, '...",None
5,25,267,"{'symmetry_breaking': True, 'mip_focus': 2, 'c...",None
6,25,153,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
7,25,153,"{'symmetry_breaking': False, 'mip_focus': 2, '...",None
8,25,153,"{'symmetry_breaking': True, 'mip_focus': 2, 'c...",None
9,25,10,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None


In [101]:
### already ran, do not run again!!

import pandas as pd
import pickle
import os
import time

# def run_VRPsolver_with_plan(run_plan_file, sleep_time=0.25):
#     """
#     Execute solver based on the run plan and save results incrementally to the same file.
    
#     Args:
#         run_plan_file (str): Path to the run plan file (source and destination).
#         sleep_time (float): Time to sleep between runs for fault tolerance.
#     """
#     # Load run plan
#     with open(run_plan_file, "rb") as f:
#         run_plan_df = pickle.load(f)

#     # Ensure ObjectiveValue column exists
#     if "ObjectiveValue" not in run_plan_df.columns:
#         run_plan_df["ObjectiveValue"] = None

#     # Iterate through runs
#     for idx, row in run_plan_df.iterrows():
#         # Skip already completed runs
#         if row["ObjectiveValue"] is not None:
#             continue
        
#         # Extract parameters
#         queue_size = row["QueueSize"]
#         problem_number = row["ProblemNumber"]
#         hyperparams = row["Hyperparameters"]

#         print(f"Running QueueSize: {queue_size}, ProblemNumber: {problem_number}, hyperparams: {hyperparams}")
        
#         # Load problem data
#         try:
#             problem_df, cost_matrix, warm_start_transition_matrix, traditional_SPT_obj_value = load_problem_data(queue_size, problem_number)

#             # Instantiate solver
#             solver = VRPSolver(
#                 cost_matrix=cost_matrix,
#                 problem_df=problem_df,
#                 problemset_num=queue_size,
#                 problem_num=problem_number,
#                 warm_start=warm_start_transition_matrix,
#                 debug=False,
#                 hyperparams={
#                     **hyperparams,  # Unpack hyperparameters
#                     "num_threads": 23,
#                 }
#             )

#             # Solve problem
#             solver.iterative_solving()
#             solution = solver.get_solution()

#             # Scoring
#             optimization_algorithm_obj_value = solution["objective_value"]
#             if optimization_algorithm_obj_value is None:
#                 improvement_relative_percent = -100  # Penalize infeasible solutions
#             else:
#                 O, T = optimization_algorithm_obj_value, traditional_SPT_obj_value
#                 improvement_relative_percent = (T - O) / max(T, 1e-4) * 100  # Avoid divide by zero

#             # Store result
#             run_plan_df.at[idx, "ObjectiveValue"] = improvement_relative_percent

#         except Exception as e:
#             print(f"Error on QueueSize: {queue_size}, ProblemNumber: {problem_number} -> {e}")
#             run_plan_df.at[idx, "ObjectiveValue"] = None  # Mark as incomplete
            
#         # Save progress after each iteration
#         with open(run_plan_file, "wb") as f:
#             pickle.dump(run_plan_df, f)

#         # Sleep between iterations
#         print("Saved! Sleeping... ZzzZ Zz...")
#         time.sleep(sleep_time)

#     print("Run completed. Results saved to run plan file.")

## RUN ONLY ONCE!
# run_VRPsolver_with_plan(run_plan_file="validation_run_plan_deterministic.pkl")


### Hold-out Testing for Deterministic Optimization

- The plan is to take every combination of QueueSize and ProblemData and generate the 25, 75 (1-300, and 1-100 permutations)
- These will be where things are stored at a high level, but solutions should also be generated for Deterministic Optimization in the non-training folder for ProblemSet Solutions
- Assumes a specific hyperparameter configuration per queue size (best params)

Given that the following are the best params for QueueSize, symmetry_breaking as False tends to be better; cuts of 3 or 2 seem to suffice but have less effect than heuristics and mip_focus.
- QueueSize: 25, hyperparams: {'symmetry_breaking': False, 'mip_focus': 1, 'cuts': 3, 'heuristics': 0.8706962488000624, 'presolve': 0, 'delta_start': 1291}
- QueueSize: 75, hyperparams: {'symmetry_breaking': False, 'mip_focus': 1, 'cuts': 2, 'heuristics': 0.7991870295315516, 'presolve': 1, 'delta_start': 2509}

In [106]:
### hold out testing already done, (Do not run again!)

import pandas as pd

# Define the range for QueueSize and ProblemNumber
QUEUE_PROBLEM_RANGE = {
    25: (1, 300),
    75: (1, 100)
}

# Define the hyperparameter sets
BEST_PARAMS = {
    25: {'symmetry_breaking': False, 'mip_focus': 1, 'cuts': 3, 'heuristics': 0.8706962488000624, 'presolve': 0, 'delta_start': 1291},
    75: {'symmetry_breaking': False, 'mip_focus': 1, 'cuts': 2, 'heuristics': 0.7991870295315516, 'presolve': 1, 'delta_start': 2509}
}

# Generate all combinations
run_plan_data = []
for queue_size, (start, end) in QUEUE_PROBLEM_RANGE.items():
    for problem_number in range(start, end + 1):
        run_plan_data.append({
            "QueueSize": queue_size,
            "ProblemNumber": problem_number,
            "Hyperparameters": BEST_PARAMS[queue_size],  # Assign hyperparameters based on QueueSize
            "ObjectiveValue": None  # Placeholder for objective value (percent improvement to Traditional_SPT objective values)
        })

# Convert to DataFrame
holdout_test_plan_deterministic = pd.DataFrame(run_plan_data)

# Display the first few rows of the DataFrame for preview
display(holdout_test_plan_deterministic)

# holdout_test_deterministic_df.to_pickle(f"holdout_test_plan_deterministic.pkl") ## COMMENTED OUT BECAUSE DO NOT NEED TO OVERWRITE


,QueueSize,ProblemNumber,Hyperparameters,ObjectiveValue
0,25,1,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
1,25,2,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
2,25,3,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
3,25,4,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
4,25,5,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
...,...,...,...,...
395,75,96,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
396,75,97,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
397,75,98,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None
398,75,99,"{'symmetry_breaking': False, 'mip_focus': 1, '...",None


In [108]:
## Already ran, do not run again

# def run_VRPsolver_with_plan(run_plan_file, training, sleep_time=0.25):
#     """
#     Execute solver based on the run plan and save results incrementally to the same file.
    
#     Args:
#         run_plan_file (str): Path to the run plan file (source and destination).
#         training (bool): Prefix for the loading of problem data (True is "Training_"; False is "")
#         sleep_time (float): Time to sleep between runs for fault tolerance.
#     """
#     # Load run plan
#     with open(run_plan_file, "rb") as f:
#         run_plan_df = pickle.load(f)

#     # Ensure ObjectiveValue column exists
#     if "ObjectiveValue" not in run_plan_df.columns:
#         run_plan_df["ObjectiveValue"] = None

#     # Iterate through runs
#     for idx, row in run_plan_df.iterrows():
#         # Skip already completed runs
#         if row["ObjectiveValue"] is not None:
#             continue
        
#         # Extract parameters
#         queue_size = row["QueueSize"]
#         problem_number = row["ProblemNumber"]
#         hyperparams = row["Hyperparameters"]

#         print(f"Running QueueSize: {queue_size}, ProblemNumber: {problem_number}, hyperparams: {hyperparams}")
        
#         # Load problem data
#         try:
#             problem_df, cost_matrix, warm_start_transition_matrix, traditional_SPT_obj_value = load_problem_data(queue_size, problem_number, training=training)

#             # Instantiate solver
#             solver = VRPSolver(
#                 cost_matrix=cost_matrix,
#                 problem_df=problem_df,
#                 problemset_num=queue_size,
#                 problem_num=problem_number,
#                 warm_start=warm_start_transition_matrix,
#                 debug=False,
#                 hyperparams={
#                     **hyperparams,  # Unpack hyperparameters  (very important!)
#                     "num_threads": 23,
#                 }
#             )

#             # Solve problem
#             solver.iterative_solving()
#             solution = solver.get_solution()

#             # Scoring
#             optimization_algorithm_obj_value = solution["objective_value"]
#             if optimization_algorithm_obj_value is None:
#                 improvement_relative_percent = -100  # Penalize infeasible solutions
#             else:
#                 O, T = optimization_algorithm_obj_value, traditional_SPT_obj_value
#                 improvement_relative_percent = (T - O) / max(T, 1e-4) * 100  # Avoid divide by zero

#             # Store result
#             run_plan_df.at[idx, "ObjectiveValue"] = improvement_relative_percent

#         except Exception as e:
#             print(f"Error on QueueSize: {queue_size}, ProblemNumber: {problem_number} -> {e}")
#             run_plan_df.at[idx, "ObjectiveValue"] = None  # Mark as incomplete
            
#         # Save progress after each iteration
#         with open(run_plan_file, "wb") as f:
#             pickle.dump(run_plan_df, f)

#         # Sleep between iterations
#         print("Saved! Sleeping... ZzzZ Zz...")
#         time.sleep(sleep_time)

#     print("Run completed. Results saved to run plan file.")

# already ran
# run_VRPsolver_with_plan(run_plan_file="holdout_test_plan_deterministic.pkl", 
#                         training =  False) # Training is False

Running QueueSize: 25, ProblemNumber: 1, hyperparams: {'symmetry_breaking': False, 'mip_focus': 1, 'cuts': 3, 'heuristics': 0.8706962488000624, 'presolve': 0, 'delta_start': 1291}
file_path used:ProblemSets/Queue25
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 25, ProblemNumber: 2, hyperparams: {'symmetry_breaking': False, 'mip_focus': 1, 'cuts': 3, 'heuristics': 0.8706962488000624, 'presolve': 0, 'delta_start': 1291}
file_path used:ProblemSets/Queue25
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 25, ProblemNumber: 3, hyperparams: {'symmetry_breaking': False, 'mip_focus': 1, 'cuts': 3, 'heuristics': 0.8706962488000624, 'presolve': 0, 'delta_start': 1291}
file_path used:ProblemSets/Queue25
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 25, ProblemNumber: 4, hyperparams: {'symmetry_breaking': False, 'mip_focus': 1, 'cuts': 3, 'heuristics': 0.8706962488000624, 'presolve': 0, 'delta_start': 1291}
file_path used:ProblemSets/Queue25
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 2

##### The above took around 36 hours to run

### Hyperparameter Tuning for Stochastic Optimization

In [119]:
# stochastic hyperparameter search
if __name__ == "__main__":
    run_combined_AFK_cycle(
        study_func=parallel_study_stochastic,  # Replace with your actual study function
        queue_sizes=[25, 25, 75],   # List of QueueSizes to cycle through (balancing out the 25's to be more than 75)
        n_trials=2,                 # Number of trials per run
        folder="optuna_studies_stochastic",    # Folder to save study results
        retry_delay=7,             # Wait 14 seconds on failure
        run_delay=0.25                 # Wait 0.25 seconds between successful runs
    )

[I 2024-11-21 12:44:04,136] A new study created in memory with name: Queue25_Study


QueueSize: 25; ProblemNumber: 192
file_path used:ProblemSets/Training_Queue25


[I 2024-11-21 12:46:14,006] Trial 0 finished with value: 16.24 and parameters: {'delta_start': 1819, 'mip_focus': 2, 'cuts': 3, 'heuristics': 0.8738974681730876, 'presolve': 0, 'nanmean': True}. Best is trial 0 with value: 16.24.


QueueSize: 25; ProblemNumber: 227
file_path used:ProblemSets/Training_Queue25


[I 2024-11-21 12:48:07,500] Trial 1 finished with value: 29.398928783971435 and parameters: {'delta_start': 1628, 'mip_focus': 0, 'cuts': 2, 'heuristics': 0.9336883946983141, 'presolve': 2, 'nanmean': True}. Best is trial 1 with value: 29.398928783971435.


...

QueueSize: 75; ProblemNumber: 23
file_path used:ProblemSets/Training_Queue75


[I 2024-11-22 12:37:52,376] Trial 183 finished with value: 12.98472385428907 and parameters: {'delta_start': 2720, 'mip_focus': 0, 'cuts': 3, 'heuristics': 0.87413593855897, 'presolve': 0, 'nanmean': False}. Best is trial 166 with value: 23.037740217642973.


Study saved to optuna_studies_stochastic/Queue75_Study.pkl
Sleeping zZz  zZz  Zzz
QueueSize: 25; ProblemNumber: 257
file_path used:ProblemSets/Training_Queue25


[W 2024-11-22 12:38:27,831] Trial 368 failed with parameters: {'delta_start': 1276, 'mip_focus': 0, 'cuts': 3, 'heuristics': 0.8438167010332456, 'presolve': 1, 'nanmean': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/jblitz/miniconda3/envs/MathOpt/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_593762/1789636899.py", line 284, in <lambda>
    lambda trial: optuna_objective_stochastic(queue_size)(trial),
  File "/tmp/ipykernel_593762/1789636899.py", line 184, in objective
    solver.stochastic_iterative_solving()
  File "/tmp/ipykernel_593762/3146046974.py", line 468, in stochastic_iterative_solving
    self.solve()
  File "/tmp/ipykernel_593762/17195182.py", line 322, in solve
    self.result = solver.solve(
  File "/home/jblitz/miniconda3/envs/MathOpt/lib/python3.10/site-packages/pyomo/opt/base/solvers.py", line 579, in solve
    _sta

[INFO] Interrupted by user. Exiting.


#### Validations for Stochastic Optimization

- Just like with deterministic optimization, create the run plan (with a learning adjustment on the `delta_start_min_by_queue_size`)

In [122]:
validation_samples

{25: array([204, 267, 153,  10, 234, 227, 197, 110,   6, 176]),
 75: array([ 3, 34, 68, 97, 51, 84, 89, 77, 66, 41])}

In [12]:
## Run once only
import pandas as pd
import pickle
import os

# Define minimum delta_start for each queue size
delta_start_min_by_queue_size = {
    25: 756, # adjusted based on results from Optuna
    75: 2495
}

# Load studies function
def load_study(folder, name):
    with open(os.path.join(folder, f"{name}.pkl"), "rb") as f:
        return pickle.load(f)

# Load studies
folder_path = "optuna_studies_stochastic"
study_names = ["Queue25_Study", "Queue75_Study"]
studies = {name: load_study(folder_path, name) for name in study_names}

# Extract trial data into a Pandas DataFrame
all_trials_data = []
for study_name, study in studies.items():
    queue_size = int(study_name.split("Queue")[1].split("_")[0])
    delta_start_min = delta_start_min_by_queue_size.get(queue_size, 0)

    for trial in study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE]):
        # Exclude 'delta_start' from the hyperparameters
        hyperparameters = {k: v for k, v in trial.params.items() if k != "delta_start"}
        all_trials_data.append({
            "StudyName": study_name,
            "QueueSize": queue_size,
            "TrialNumber": trial.number,
            "DeltaStart": trial.params.get("delta_start", 0),
            "ObjectiveValue": trial.values[0] if trial.values else None,
            "Hyperparameters": hyperparameters  # Store hyperparameters as a dict
        })

# Convert to DataFrame
trials_df = pd.DataFrame(all_trials_data)

# Filter by delta_start_min and valid ObjectiveValue
filtered_trials_df = trials_df[
    (trials_df["DeltaStart"] > trials_df["QueueSize"].map(delta_start_min_by_queue_size))
]

# Sort and group by queue size
sorted_trials_df = (
    filtered_trials_df.sort_values(by=["QueueSize", "ObjectiveValue"], ascending=[True, False])
)

# Select top 3 rows per StudyName (Stochastic)
print(f"Table for Top 3 Best Params for Stochastic Optimization Validation")
top_3_per_study = sorted_trials_df.groupby("StudyName").head(3).copy()
min_delta_start_per_queue = top_3_per_study.groupby("QueueSize")["DeltaStart"].transform("min") # per row
top_3_per_study["DeltaStart_adjusted"] = min_delta_start_per_queue
top_3_per_study["Hyperparameters"] = top_3_per_study.apply(
    lambda row: {**row["Hyperparameters"], "delta_start": row["DeltaStart_adjusted"]}, axis=1
)

display(top_3_per_study)


Table for Top 3 Best Params for Stochastic Optimization Validation


,StudyName,QueueSize,TrialNumber,DeltaStart,ObjectiveValue,Hyperparameters,DeltaStart_adjusted
102,Queue25_Study,25,102,1774,46.514066,"{'mip_focus': 3, 'cuts': 3, 'heuristics': 0.86...",818
186,Queue25_Study,25,186,818,43.369673,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.76...",818
269,Queue25_Study,25,269,1348,43.369673,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.75...",818
534,Queue75_Study,75,166,2952,23.037740,"{'mip_focus': 1, 'cuts': 2, 'heuristics': 0.58...",2619
407,Queue75_Study,75,39,2619,19.897338,"{'mip_focus': 1, 'cuts': 3, 'heuristics': 0.96...",2619
441,Queue75_Study,75,73,2957,19.810336,"{'mip_focus': 1, 'cuts': 2, 'heuristics': 0.72...",2619


In [125]:
### ONLY RUN ONCE!!! Do not overwrite
### run plan for stochastic (recycling the top_3_per_study for stochastic)
import pandas as pd
import itertools
import pickle

# Generate run plan data
run_plan_data = []

# Iterate over queue sizes and problems
for queue_size, problems in validation_samples.items():
    # Filter top hyperparameters
    top_3_hyperparams = top_3_per_study[top_3_per_study["QueueSize"] == queue_size]["Hyperparameters"] # uses same variables
    
    # Combine problems and hyperparameters
    for problem_number, hyperparams in itertools.product(problems, top_3_hyperparams):
        run_plan_data.append({
            "QueueSize": queue_size,  # Add queue size
            "ProblemNumber": problem_number,  # Add problem number
            "Hyperparameters": hyperparams,  # Add hyperparameter set
            "ObjectiveValue": None  # Placeholder for results
        })

# Convert to DataFrame
run_plan_df = pd.DataFrame(run_plan_data)

# Save run plan
run_plan_file = "validation_run_plan_stochastic.pkl" # ensure it doesn't save over deterministic

# with open(run_plan_file, "wb") as f: ## Commented out so it doesn't overwrite!!!!!!!!!!!
#     pickle.dump(run_plan_df, f)

# Display the run plan|
display(run_plan_df)


,QueueSize,ProblemNumber,Hyperparameters,ObjectiveValue
0,25,204,"{'mip_focus': 3, 'cuts': 3, 'heuristics': 0.86...",None
1,25,204,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.76...",None
2,25,204,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.75...",None
3,25,267,"{'mip_focus': 3, 'cuts': 3, 'heuristics': 0.86...",None
4,25,267,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.76...",None
5,25,267,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.75...",None
6,25,153,"{'mip_focus': 3, 'cuts': 3, 'heuristics': 0.86...",None
7,25,153,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.76...",None
8,25,153,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.75...",None
9,25,10,"{'mip_focus': 3, 'cuts': 3, 'heuristics': 0.86...",None


In [131]:
import pandas as pd
import pickle
import os
import time

def run_VRPsolver_with_plan(run_plan_file, sleep_time=0.25):
    """
    Execute solver based on the run plan and save results incrementally to the same file.
    
    Args:
        run_plan_file (str): Path to the run plan file (source and destination).
        sleep_time (float): Time to sleep between runs for fault tolerance.
    """
    # Load run plan
    with open(run_plan_file, "rb") as f:
        run_plan_df = pickle.load(f)

    # Ensure ObjectiveValue column exists
    if "ObjectiveValue" not in run_plan_df.columns:
        run_plan_df["ObjectiveValue"] = None

    # Iterate through runs
    for idx, row in run_plan_df.iterrows():
        # Skip already completed runs
        if row["ObjectiveValue"] is not None:
            continue
        
        # Extract parameters
        queue_size = row["QueueSize"]
        problem_number = row["ProblemNumber"]
        hyperparams = row["Hyperparameters"]

        print(f"Running QueueSize: {queue_size}, ProblemNumber: {problem_number}, hyperparams: {hyperparams}")
        
        # Load problem data
        try:
            problem_df, cost_matrix, warm_start_transition_matrix, traditional_SPT_obj_value = load_problem_data(queue_size, problem_number, training=True) # using training data still

            # Instantiate solver
            solver = StochasticVRPSolver(
                cost_matrix=cost_matrix,
                problem_df=problem_df,
                problemset_num=queue_size,
                problem_num=problem_number,
                warm_start=warm_start_transition_matrix,
                debug=False,
                stochastic_parameters={
                    "nanmean": hyperparams.get("nanmean"),  # Dynamically retrieve nanmean
                },
                hyperparams={
                    **hyperparams,  # unpack hyperparameters
                    "time_limit": 10, # seconds
                    "symmetry_breaking": False, # based on learnings from deterministic optimization
                    "num_threads": 23,
                }
            )
    
            # Solve problem
            solver.stochastic_iterative_solving() # critically needs to use the stochastic solving features
            solution = solver.get_solution() # answers always result in deterministic solves, because next job is null

            # Scoring
            optimization_algorithm_obj_value = solution["objective_value"]
            if optimization_algorithm_obj_value is None:
                improvement_relative_percent = -100  # Penalize infeasible solutions
            else:
                O, T = optimization_algorithm_obj_value, traditional_SPT_obj_value
                improvement_relative_percent = (T - O) / max(T, 1e-4) * 100  # Avoid divide by zero

            # Store result
            run_plan_df.at[idx, "ObjectiveValue"] = improvement_relative_percent

        except Exception as e:
            print(f"Error on QueueSize: {queue_size}, ProblemNumber: {problem_number} -> {e}")
            run_plan_df.at[idx, "ObjectiveValue"] = None  # Mark as incomplete
            
        # Save progress after each iteration
        with open(run_plan_file, "wb") as f:
            pickle.dump(run_plan_df, f)

        # Sleep between iterations
        print("Saved! Sleeping... ZzzZ Zz...")
        time.sleep(sleep_time)

    print("Run completed. Results saved to run plan file.")

# RUN ONLY ONCE!
run_VRPsolver_with_plan(run_plan_file="validation_run_plan_stochastic.pkl")


Running QueueSize: 25, ProblemNumber: 204, hyperparams: {'mip_focus': 3, 'cuts': 3, 'heuristics': 0.8678944201458589, 'presolve': 1, 'nanmean': False, 'delta_start': 818}
file_path used:ProblemSets/Training_Queue25
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 25, ProblemNumber: 204, hyperparams: {'mip_focus': 0, 'cuts': 3, 'heuristics': 0.7684551530323259, 'presolve': 1, 'nanmean': False, 'delta_start': 818}
file_path used:ProblemSets/Training_Queue25
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 25, ProblemNumber: 204, hyperparams: {'mip_focus': 0, 'cuts': 3, 'heuristics': 0.7563919824960467, 'presolve': 1, 'nanmean': False, 'delta_start': 818}
file_path used:ProblemSets/Training_Queue25
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 25, ProblemNumber: 267, hyperparams: {'mip_focus': 3, 'cuts': 3, 'heuristics': 0.8678944201458589, 'presolve': 1, 'nanmean': False, 'delta_start': 818}
file_path used:ProblemSets/Training_Queue25
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 2

### Hold-out Testing for Stochastic Optimization

In [133]:
### ADAPT FOR STOCHASTIC OPTIMIZATION (runs only once to initiate)

import pandas as pd

# Define the range for QueueSize and ProblemNumber
QUEUE_PROBLEM_RANGE = {
    25: (1, 300),
    75: (1, 100)
}

# Define the hyperparameter sets
BEST_PARAMS = {  # change to the newest best per queue size
    25: {'mip_focus': 0, 'cuts': 3, 'heuristics': 0.7684551530323259, 'presolve': 1, 'nanmean': False, 'delta_start': 818},
    75: {'mip_focus': 1, 'cuts': 2, 'heuristics': 0.5851062188632922, 'presolve': 1, 'nanmean': False, 'delta_start': 2619}
}

# Generate all combinations
run_plan_data = []
for queue_size, (start, end) in QUEUE_PROBLEM_RANGE.items():
    for problem_number in range(start, end + 1):
        run_plan_data.append({
            "QueueSize": queue_size,
            "ProblemNumber": problem_number,
            "Hyperparameters": BEST_PARAMS[queue_size],  # Assign hyperparameters based on QueueSize
            "ObjectiveValue": None  # Placeholder for objective value (percent improvement to Traditional_SPT objective values)
        })

# Convert to DataFrame
holdout_test_plan_stochastic = pd.DataFrame(run_plan_data)

# Display the first few rows of the DataFrame for preview
display(holdout_test_plan_stochastic)

#holdout_test_plan_stochastic.to_pickle(f"holdout_test_plan_stochastic.pkl") ### commented out, do not overwrite


,QueueSize,ProblemNumber,Hyperparameters,ObjectiveValue
0,25,1,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.76...",None
1,25,2,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.76...",None
2,25,3,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.76...",None
3,25,4,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.76...",None
4,25,5,"{'mip_focus': 0, 'cuts': 3, 'heuristics': 0.76...",None
...,...,...,...,...
395,75,96,"{'mip_focus': 1, 'cuts': 2, 'heuristics': 0.58...",None
396,75,97,"{'mip_focus': 1, 'cuts': 2, 'heuristics': 0.58...",None
397,75,98,"{'mip_focus': 1, 'cuts': 2, 'heuristics': 0.58...",None
398,75,99,"{'mip_focus': 1, 'cuts': 2, 'heuristics': 0.58...",None


In [11]:
# adapt for STOCHASTIC OPTIMIZATION!!

import pandas as pd
import pickle
import os
import time

def run_VRPsolver_with_plan(run_plan_file, training, sleep_time=0.25):
    """
    Execute solver based on the run plan and save results incrementally to the same file.
    
    Args:
        run_plan_file (str): Path to the run plan file (source and destination).
        training (bool): Prefix for the loading of problem data (True is "Training_"; False is "")
        sleep_time (float): Time to sleep between runs for fault tolerance.
    """
    # Load run plan
    with open(run_plan_file, "rb") as f:
        run_plan_df = pickle.load(f)

    # Ensure ObjectiveValue column exists
    if "ObjectiveValue" not in run_plan_df.columns:
        run_plan_df["ObjectiveValue"] = None

    # Iterate through runs
    for idx, row in run_plan_df.iterrows():
        # Skip already completed runs
        if row["ObjectiveValue"] is not None:
            continue
        
        # Extract parameters
        queue_size = row["QueueSize"]
        problem_number = row["ProblemNumber"]
        hyperparams = row["Hyperparameters"]

        print(f"Running QueueSize: {queue_size}, ProblemNumber: {problem_number}, hyperparams: {hyperparams}")
        
        # Load problem data
        try:
            problem_df, cost_matrix, warm_start_transition_matrix, traditional_SPT_obj_value = load_problem_data(queue_size, problem_number, training=training)

            # Instantiate solver
            solver = StochasticVRPSolver(
                cost_matrix=cost_matrix,
                problem_df=problem_df,
                problemset_num=queue_size,
                problem_num=problem_number,
                warm_start=warm_start_transition_matrix,
                debug=False,
                stochastic_parameters={
                    "nanmean": hyperparams.get("nanmean"),  # Dynamically retrieve nanmean
                },
                hyperparams={
                    **hyperparams,  # unpack hyperparameters
                    "time_limit": 10, # seconds
                    "symmetry_breaking": False, # based on learnings from deterministic optimization
                    "num_threads": 23,
                }
            )

            # Solve problem
            solver.stochastic_iterative_solving() # critically needs to use the stochastic solving features
            solution = solver.get_solution() # answers always result in deterministic solves, because next job is null

            # Scoring
            optimization_algorithm_obj_value = solution["objective_value"]
            if optimization_algorithm_obj_value is None:
                improvement_relative_percent = -100  # Penalize infeasible solutions
            else:
                O, T = optimization_algorithm_obj_value, traditional_SPT_obj_value
                improvement_relative_percent = (T - O) / max(T, 1e-4) * 100  # Avoid divide by zero

            # Store result
            run_plan_df.at[idx, "ObjectiveValue"] = improvement_relative_percent

        except Exception as e:
            print(f"Error on QueueSize: {queue_size}, ProblemNumber: {problem_number} -> {e}")
            run_plan_df.at[idx, "ObjectiveValue"] = None  # Mark as incomplete
            
        # Save progress after each iteration
        with open(run_plan_file, "wb") as f:
            pickle.dump(run_plan_df, f)

        # Sleep between iterations
        print("Saved! Sleeping... ZzzZ Zz...")
        time.sleep(sleep_time)

    print("Run completed. Results saved to run plan file.")

run_VRPsolver_with_plan(run_plan_file="holdout_test_plan_stochastic.pkl",  ## CAREFUL HERE
                        training =  False) # Training is False

Running QueueSize: 75, ProblemNumber: 45, hyperparams: {'mip_focus': 1, 'cuts': 2, 'heuristics': 0.5851062188632922, 'presolve': 1, 'nanmean': False, 'delta_start': 2619}
file_path used:ProblemSets/Queue75
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 75, ProblemNumber: 46, hyperparams: {'mip_focus': 1, 'cuts': 2, 'heuristics': 0.5851062188632922, 'presolve': 1, 'nanmean': False, 'delta_start': 2619}
file_path used:ProblemSets/Queue75
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 75, ProblemNumber: 47, hyperparams: {'mip_focus': 1, 'cuts': 2, 'heuristics': 0.5851062188632922, 'presolve': 1, 'nanmean': False, 'delta_start': 2619}
file_path used:ProblemSets/Queue75
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 75, ProblemNumber: 48, hyperparams: {'mip_focus': 1, 'cuts': 2, 'heuristics': 0.5851062188632922, 'presolve': 1, 'nanmean': False, 'delta_start': 2619}
file_path used:ProblemSets/Queue75
Saved! Sleeping... ZzzZ Zz...
Running QueueSize: 75, ProblemNumber: 49, hyperparams: {

##### The above took 36 hours to evaluate

## Debugging capability with porting MILP model
- To debug and ensure proper porting of the math optimization model, I would export the decision matrices and test them to see how they compared to an Excel Solverstudio version. This served as my baseline checker and helped with additions to debugging constraints.

In [67]:
# Lambda function to save each k slice to CSV (for testing out in Excel model)
save_decision_slices = lambda decisions: [
    # Loop over k to slice decisions array
    pd.DataFrame(decisions[:, :, k])  # Convert kth slice to DataFrame
    .to_csv(f'decision_xij{k + 1}.csv', index=False)  # Save DataFrame to CSV
    for k in range(decisions.shape[2])  # Loop over k dimension
]



save_decision_slices(solution['decision_matrix'])


[None, None]

In [189]:
def export_cost_matrices_to_csv_with_ab(cost_matrix, a, b, filename='output.csv'):
    """
    Exports 3D cost matrices and 1D vectors a and b to a CSV file.
    
    Parameters:
    - cost_matrix (np.array): A 3D numpy array with dimensions (N, N, k) representing cost matrices.
    - a (np.array): A 1D numpy array representing vector a.
    - b (np.array): A 1D numpy array representing vector b.
    - filename (str): Output CSV file name.
    """

    # Ensure dimensions for a and b are consistent
    assert a.shape[0] == b.shape[0], "Arrays 'a' and 'b' must have the same length."

    # Open the CSV file to write
    with open(filename, 'w') as file:
        
        # Iterate through each k dimension (cost matrix slice)
        for k in range(cost_matrix.shape[2]):
            # Write the label for each cost matrix slice
            file.write(f"k={k}\n")
            
            # Write each row of the cost matrix slice for the current k
            np.savetxt(file, cost_matrix[:, :, k], fmt='%d', delimiter=',')
            file.write("\n")  # Separate slices with a newline

        # Write labels for a and b
        file.write("a,b\n")

        # Concatenate a and b horizontally and write them to the file
        ab_combined = np.column_stack((a, b))
        np.savetxt(file, ab_combined, fmt='%d', delimiter=',')
        print("Saved file")

In [190]:
export_cost_matrices_to_csv_with_ab(cost_matrix, a, b)

Saved file
